<a href="https://colab.research.google.com/github/snf5/Analisis-de-Datos-Predictivo-en-el-Sector-Turistico/blob/main/Airbnb_Torrevieja_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATOS A ANALIZAR SOBRE EL SECTOR TURÍSTICA EN TORREVIEJA (RESERVAS A TRAVÉS DE AIRBNB Y VRBO) JUNTO CON LOS EVENTOS ORGANIZADOS EN EL PERÍODO DE TIEMPO A ANALIZAR

In [ ]:
#IMPORTAMOS LAS LIBRERÍAS QUE VAMOSA NECESITAR A LA HORA DE TRABAJAR
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import numpy as np
from sklearn.model_selection import train_test_split

# DATOS MESES POR ALOJAMIENTO


DATOS POR MESES DE CADA ALOJAMIENTO

In [ ]:
#CARGAMOS LOS DATOS DE CADA MES PARA QUEDARNOS CON LOS APARTAMENTOS QUE NOS INTERESA ANALIZA, SOLO AQUELLOS QUE REALMENTE HAN SIDO ALQUILADOS EN EL PERÍODO ANALIZADO
datos_meses = pd.read_csv("/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/Spain-torrevieja_Monthly_Match_2023-11-23.csv")
datos_meses

In [ ]:
#ANALIZAMOS LAS COLUMAS
datos_meses.columns

In [ ]:
#ORDENAMOS PARA CONOCER EL MES INICAL A ANALIZAR
datos_meses.sort_values(by="Reporting Month")

In [ ]:
#NOS QUEDAMOS CON LAS COLUMNAS QUE NOS INTERESAS Y NOS VAN A APORTAR VALOR AL ANÁLISIS
campos_meses = ['Property ID','Reporting Month', 'Bedrooms', 'Occupancy Rate', 'Revenue (USD)', 'Revenue (Native)', 'ADR (USD)', 'ADR (Native)', 'Number of Reservations', 'Reservation Days', 'Available Days', 'Blocked Days']
datos_meses_columnas = datos_meses.loc[:, campos_meses]
datos_meses_columnas_todo = datos_meses.loc[:, campos_meses]
datos_meses_columnas

In [ ]:
#ANALIZAMOS LAS COLUMNAS
datos_meses_columnas.columns

VALORES AUSENTES MONTHLY

In [ ]:
datos_meses_columnas.groupby('Property ID').sum()

# VALORES AUSENTES MONTHLY

VALORES AUSENTES

In [ ]:
datos_meses_columnas_ausentes_todo = datos_meses_columnas.isna().sum().reset_index()
datos_meses_columnas_ausentes_todo.columns = ['Columna', 'Valores Ausentes']
datos_meses_columnas_ausdatos_meses_columnas_ausentes_todoentes = datos_meses_columnas_ausentes_todo.sort_values(by='Valores Ausentes', ascending=False)
datos_ausentes_filtrados = datos_meses_columnas_ausentes_todo[datos_meses_columnas_ausentes_todo['Valores Ausentes'] > 0]
plt.figure(figsize=(14, 8))
sns.barplot(y='Valores Ausentes', x='Columna',data=datos_ausentes_filtrados)
plt.title("Valores ausentes en dataset monthly")
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/valores_ausentes_monthly.pdf', dpi=600, format='pdf')
print(datos_ausentes_filtrados)

RELLENAMOS LOS CAMPOS NATIVE PARA ASÍ PODER TENER TODOS LOS VALORES EN LAS MISMAS UNIDADES DE EUR

In [ ]:
tasa_de_cambio = 0.955
datos_meses_columnas['Revenue (Native)'].fillna(datos_meses_columnas['Revenue (USD)'] * tasa_de_cambio, inplace=True)
datos_meses_columnas['ADR (Native)'].fillna(datos_meses_columnas['ADR (USD)'] * tasa_de_cambio, inplace=True)
datos_meses_columnas['Revenue (Native)'].round(2)
datos_meses_columnas['ADR (Native)'].round(2)
datos_meses_columnas

In [ ]:
datos_meses_columnas_ausentes_todo = datos_meses_columnas.isna().sum().reset_index()
datos_meses_columnas_ausentes_todo.columns = ['Columna', 'Valores Ausentes']
datos_meses_columnas_ausdatos_meses_columnas_ausentes_todoentes = datos_meses_columnas_ausentes_todo.sort_values(by='Valores Ausentes', ascending=False)
datos_ausentes_filtrados = datos_meses_columnas_ausentes_todo[datos_meses_columnas_ausentes_todo['Valores Ausentes'] > 0]
plt.figure(figsize=(14, 8))
sns.barplot(y='Valores Ausentes', x='Columna',data=datos_ausentes_filtrados)
plt.title("Valores ausentes en dataset monthly tras conversión de moneda")
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/valores_ausentes_monthly_tras_filtrado.pdf', dpi=600, format='pdf')
print(datos_ausentes_filtrados)

In [ ]:
datos_meses_columnas_ausentes2 = datos_meses_columnas
adr_ausentes = datos_meses_columnas_ausentes2['ADR (USD)'].isnull()
adr_ausentes_native = datos_meses_columnas_ausentes2['ADR (Native)'].isnull()
datos_meses_columnas.loc[adr_ausentes, 'ADR (USD)']=datos_meses_columnas.loc[adr_ausentes, 'Revenue (USD)'] / datos_meses_columnas.loc[adr_ausentes, 'Reservation Days']
datos_meses_columnas.loc[adr_ausentes_native, 'ADR (Native)']=datos_meses_columnas.loc[adr_ausentes_native, 'Revenue (Native)'] / datos_meses_columnas.loc[adr_ausentes_native, 'Reservation Days']

In [ ]:
datos_meses_columnas_ausentes3 = datos_meses_columnas_ausentes2.isna().sum().reset_index()
datos_meses_columnas_ausentes3.columns = ['Columna', 'Valores Ausentes']
datos_meses_columnas_ausentes3 = datos_meses_columnas_ausentes3.sort_values(by='Valores Ausentes', ascending=False)
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna',data=datos_meses_columnas_ausentes3)
plt.xticks(rotation=-90)
plt.show()

EL SIGUIENTE DATASET, SI SE AGRUPA Y SE CALCULA EL OCCUPANCY RATE BIEN SE PUEDE SUSTITUIR POR EL SUMATORIO, CALCULAMOS BIEN EL ADR Y QUITAMOS EL MONTH Y YA TENEMOS UN DATASET COMPLETO DE CADA ALOJAMIENTO

In [ ]:
datosMonthly = datos_meses_columnas[(datos_meses_columnas['Occupancy Rate'] > 0) &
 (datos_meses_columnas['Revenue (USD)'] > 0)]

In [ ]:
datosMonthly.groupby('Property ID').sum()

In [ ]:
datosMonthly

In [ ]:
datosMonthly.describe()

In [ ]:
datos_meses_columnas_ausentes3333 = datosMonthly.isna().sum().reset_index()
datos_meses_columnas_ausentes3333.columns = ['Columna', 'Valores Ausentes']
datos_meses_columnas_ausentes3333 = datos_meses_columnas_ausentes3333.sort_values(by='Valores Ausentes', ascending=False)
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna',data=datos_meses_columnas_ausentes3333)
plt.xticks(rotation=-90)
plt.show()

# DATOS DIARIOS POR ALOJAMIENTO

LISTADO POR DÍAS PARA COMPROBAR DATOS CON MONTHLY

In [ ]:
datos_diarios = pd.read_csv("/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/Spain-torrevieja_Daily_Match_2023-11-23.csv")

In [ ]:
datos_diarios

In [ ]:
datos_diarios_reservados = datos_diarios[datos_diarios['Status'] == 'R']
datos_diarios_reservados.groupby('Property ID').sum()

In [ ]:
datos_diarios_reservados

# VALORES AUSENTES DAILY

DATASET CON DATOS DIARIOS DE TODOS LOS APARTAMENTOS PRESENTES EN LOS 3 DATASETS


In [ ]:
campos_daily = ['Property ID', 'Date', 'Status', 'Price (USD)', 'Price (Native)']
datos_diarios_reservados = datos_diarios_reservados.loc[:, campos_daily]
datos_diarios_reservados

In [ ]:
datos_diarios_columnas_ausentes = datos_diarios_reservados.isna().sum().reset_index()
datos_diarios_columnas_ausentes.columns = ['Columna', 'Valores Ausentes']
datos_diarios_columnas_ausentes = datos_diarios_columnas_ausentes.sort_values(by='Valores Ausentes', ascending=False)
datos_diarios_columnas_ausentes_filtrado = datos_diarios_columnas_ausentes[datos_diarios_columnas_ausentes['Valores Ausentes'] > 0]
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna',data=datos_diarios_columnas_ausentes_filtrado)
plt.title("Valores ausentes dataset daily")
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/valores_ausentes_daily.pdf', dpi=600, format='pdf')

In [ ]:
tasa_de_cambio = 0.955
datos_diarios_reservados['Price (Native)'].fillna(datos_diarios_reservados['Price (USD)'] * tasa_de_cambio, inplace=True)
datos_diarios_reservados

In [ ]:
datos_diarios_columnas_ausentes = datos_diarios_reservados[['Property ID', 'Date', 'Status', 'Price (USD)', 'Price (Native)']].isna().sum().reset_index()
datos_diarios_columnas_ausentes.columns = ['Columna', 'Valores Ausentes']
datos_diarios_columnas_ausentes = datos_diarios_columnas_ausentes.sort_values(by='Valores Ausentes', ascending=False)
datos_diarios_columnas_ausentes_filtrado2 = datos_diarios_columnas_ausentes[datos_diarios_columnas_ausentes['Valores Ausentes'] > 0]
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna',data=datos_diarios_columnas_ausentes_filtrado2)
plt.title("Valores ausentes dataset daily tras conversión de moneda")
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/valores_ausentes_daily_filtrados.pdf', dpi=600, format='pdf')

In [ ]:
datos_diarios_reservados_unicos_sin_ausentes = datos_diarios_reservados.dropna(subset=['Price (Native)'])

In [ ]:
datos_diarios_columnas_ausentes22 = datos_diarios_reservados_unicos_sin_ausentes[['Property ID', 'Date', 'Status', 'Price (USD)', 'Price (Native)']].isna().sum().reset_index()
datos_diarios_columnas_ausentes22.columns = ['Columna', 'Valores Ausentes']
datos_diarios_columnas_ausentes22 = datos_diarios_columnas_ausentes22.sort_values(by='Valores Ausentes', ascending=False)
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna',data=datos_diarios_columnas_ausentes22)
plt.xticks(rotation=-90)
plt.show()

In [ ]:
datos_diarios_reservados.describe()

In [ ]:
datos_diarios_reservados[['Property ID','Price (USD)', 'Price (Native)']].groupby('Property ID').sum()
datos_filtrados222 = datos_diarios_reservados[(datos_diarios_reservados['Price (USD)'] > 1) & (datos_diarios_reservados['Price (Native)'] > 1)]
datos_filtrados222[['Property ID','Price (USD)', 'Price (Native)']].groupby('Property ID').sum()

In [ ]:
datos_filtrados222.describe()

In [ ]:
datosDaily = datos_filtrados222
datosTodos = datosDaily.describe()
descriptivas_sin_mean = datosTodos.drop('min')
descriptivas_sin_mean

In [ ]:
indice_menor_precio = datosDaily['Price (USD)'].idxmin()
propiedad_menor_precio = datosDaily.loc[indice_menor_precio, 'Property ID']
propiedad_menor_precio

In [ ]:
datosDaily[datosDaily['Property ID'] == 'ha-10154190ha'].describe()

In [ ]:
#le añado una columna con un valor de conteo
datos_diarios_reservados_unicos_sin_ausentes = datos_filtrados222
datos_diarios_reservados_unicos_sin_ausentes['conteo'] = 1
#los agrupo por property id
datos_diarios_reservados_unicos_conteo = datos_diarios_reservados_unicos_sin_ausentes.groupby('Property ID').sum().reset_index()
#ahora tenemos el total de ingresos de cada apartamento y el total de días de cada apartamento

#creamos columna para sacar el precio medio de alquiler de cada apartamento
datos_diarios_reservados_unicos_conteo['Precio Medio'] = datos_diarios_reservados_unicos_conteo['Price (Native)'] / datos_diarios_reservados_unicos_conteo['conteo']
datos_diarios_reservados_unicos_conteo[['Property ID', 'Precio Medio']]

In [ ]:
datos_diarios_reservados_unicos_conteo.describe()

In [ ]:
datos_finales_finales_properties = datos_diarios_reservados_unicos_conteo['Property ID']
datos_finales_finales_properties

In [ ]:
solo_precio = datos_diarios_reservados_unicos_conteo[['Property ID', 'Precio Medio']]
solo_precio

# DATOS DE CADA ALOJAMIENTO

DATOS DE CADA UNO DE LOS ALOJAMIENTOS

In [ ]:
datos_propiedades = pd.read_csv("/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/Spain-torrevieja_Property_Extended_Match_2023-11-23.csv")
datos_propiedades

In [ ]:
valores_unicos = datos_propiedades['Cancellation Policy'].unique()
valores_unicos

In [ ]:
prueba_cleaning_fee = datos_propiedades[['Property ID','Cleaning Fee (USD)']]

In [ ]:
datos_propiedades.columns

In [ ]:
estancia_minima = datos_propiedades[['Property ID','Minimum Stay']]

# VALORES AUSENTES PROPERTY

VALORES AUSENTES PROPERTY

In [ ]:
valores_ausentes_property = datos_propiedades.isna().sum().reset_index()
valores_ausentes_property.columns = ['Columna', 'Valores Ausentes']
valores_ausentes_property = valores_ausentes_property[(valores_ausentes_property['Valores Ausentes'] > 0) & (valores_ausentes_property['Valores Ausentes'] <= 10000)]
valores_ausentes_property = valores_ausentes_property.sort_values(by='Valores Ausentes', ascending=False)
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna',data=valores_ausentes_property)
plt.xticks(rotation=-90)
plt.show()

In [ ]:
#NOS QUEDAMOS CON LAS COLUMNAS QUE NOS APORTAN VALOR AL ANÁLISIS
campos_properties = ['Property ID', 'Property Type','Listing Type', 'Created Date', 'Occupancy Rate LTM', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Response Rate', 'Extra People Fee (USD)', 'Number of Photos', 'Latitude', 'Longitude', 'Amenities', 'Minimum Stay', 'Cancellation Policy', 'Pets Allowed']

In [ ]:
datos_properties_columnas = datos_propiedades.loc[:, campos_properties]
datos_properties_columnas

In [ ]:
datos_properties_columnas2 = datos_properties_columnas
datos_properties_columnas2[(datos_properties_columnas2['Bedrooms'] > 0) & (datos_properties_columnas2['Bathrooms'] > 0) & (datos_properties_columnas2['Max Guests'] > 0)]
datos_properties_columnas2

In [ ]:
datos_properties_columnas[(datos_properties_columnas['Bedrooms'] == 0) | (datos_properties_columnas['Bathrooms'] == 0)]

In [ ]:
valores_ausentes_property_columnas = datos_properties_columnas.isna().sum().reset_index()
valores_ausentes_property_columnas.columns = ['Columna', 'Valores Ausentes']
valores_ausentes_property_columnas = valores_ausentes_property_columnas[valores_ausentes_property_columnas['Valores Ausentes'] > 0]
valores_ausentes_property_columnas = valores_ausentes_property_columnas.sort_values(by='Valores Ausentes', ascending=False)
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna', data=valores_ausentes_property_columnas)
plt.title('Valores ausentes en dataset property')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/valores_ausentes_property.pdf', dpi=300, format='pdf')
print(valores_ausentes_property_columnas)

In [ ]:
datos_properties_columnas_completadas = datos_properties_columnas
datos_properties_columnas_completadas['Property Type'].fillna('Apartment', inplace=True)
datos_properties_columnas_completadas['Listing Type'].fillna('Entire home/apt', inplace=True)
datos_properties_columnas_completadas = datos_properties_columnas_completadas[datos_properties_columnas_completadas['Number of Photos'].notnull()]

valores_ausentes_property_columnas_completadas = datos_properties_columnas_completadas.isna().sum().reset_index()
valores_ausentes_property_columnas_completadas.columns = ['Columna', 'Valores Ausentes']
valores_ausentes_property_columnas_completadas = valores_ausentes_property_columnas_completadas[valores_ausentes_property_columnas_completadas['Valores Ausentes'] > 0]
valores_ausentes_property_columnas_completadas = valores_ausentes_property_columnas_completadas.sort_values(by='Valores Ausentes', ascending=False)
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna', data=valores_ausentes_property_columnas_completadas)
plt.title('Valores ausentes en dataset property')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/valores_ausentes_property.pdf', dpi=300, format='pdf')
print(valores_ausentes_property_columnas_completadas)

# MODIFICACION DE DATOS, GENERACIÓN DE NUEVAS COLUMNAS Y ELIMINACION DE DATOS INCONGRUENTES

ELIMINAMOS ALOJAMIENTOS CON 0 BAÑOS Y 0 HABITACIONES

AHORA TENEMOS EL DATASET datos_properties_columnas_completadas EL CUAL ES EL QUE TIENE VALORES COMPLETADOS

In [ ]:
alojamientos_cero = datos_propiedades[
    (datos_propiedades['Bedrooms'] == 0) |
    (datos_propiedades['Bathrooms'] == 0) |
    (datos_propiedades['Max Guests'] == 0)
]
resumen = alojamientos_cero[['Bedrooms', 'Bathrooms', 'Max Guests']].describe()
resumen

In [ ]:
datos_propiedades[datos_propiedades['Bathrooms'] == 0]

In [ ]:
datos_propiedades[datos_propiedades['Bedrooms'] == 0]

In [ ]:
datos_propiedades[datos_propiedades['Max Guests'] == 0]

In [ ]:
datos_properties_columnas_completadas = datos_properties_columnas_completadas[(datos_properties_columnas_completadas['Bathrooms'] != 0) & (datos_properties_columnas_completadas['Bedrooms'] != 0) & (datos_properties_columnas_completadas['Max Guests'] != 0)]

DATASET MODIFICADO Y TRANSFORMADO CON TODAS LAS VARIABLES NECESARIAS A ANALIZAR

AÑADIMOS 1 O 0 EN CASO DE QUE HAYA EXTRA PEOPLE FEE

In [ ]:
transformacion = lambda x: 1 if pd.notnull(x) else 0
datos_properties_columnas_completadas['Extra People Fee (USD)'] = datos_properties_columnas_completadas['Extra People Fee (USD)'].apply(transformacion)
datos_properties_columnas_completadas

AÑADIMOS AÑO DE CREACIÓN EN LAS PLATAFORMAS

In [ ]:
fecha_minima_por_alojamiento = datos_diarios.groupby('Property ID')['Date'].min().reset_index()
fecha_minima_por_alojamiento.columns = ['Property ID', 'Fecha Minima']
fecha_minima_por_alojamiento

In [ ]:
datos_properties_columnas_completadas = pd.merge(datos_properties_columnas_completadas,fecha_minima_por_alojamiento , on='Property ID', how='left')
datos_properties_columnas_completadas['Created Date'] = datos_properties_columnas_completadas['Created Date'].fillna(datos_properties_columnas_completadas['Fecha Minima'])
datos_properties_columnas_completadas = datos_properties_columnas_completadas.drop(columns=['Fecha Minima'])
datos_properties_columnas_completadas

In [ ]:
datos_properties_columnas_completadas['Created Date'] = pd.to_datetime(datos_properties_columnas_completadas['Created Date'])
datos_properties_columnas_completadas['Año Creacion'] = datos_properties_columnas_completadas['Created Date'].dt.strftime('%Y')
datos_properties_columnas_completadas

AÑADIMOS COLUMNAS BOOLEANAS DE SERVICIOS MAS DESTACADOS QUE OFREZCAN LOS ALOJAMIENTOS

In [ ]:
#Añadimos pool, elevator/lift, parking, internet, tv, private pool, aire acondicionado
datos_properties_columnas_completadas['Pool'] = 0
datos_properties_columnas_completadas.loc[datos_properties_columnas_completadas['Amenities'].str.contains('pool', case=False), 'Pool'] = 1

datos_properties_columnas_completadas['Lift/Elevator'] = 0
datos_properties_columnas_completadas.loc[datos_properties_columnas_completadas['Amenities'].str.contains('lift|elevator', case=False), 'Lift/Elevator'] = 1

datos_properties_columnas_completadas['Parking'] = 0
datos_properties_columnas_completadas.loc[datos_properties_columnas_completadas['Amenities'].str.contains('parking', case=False), 'Parking'] = 1

datos_properties_columnas_completadas['Internet'] = 0
datos_properties_columnas_completadas.loc[datos_properties_columnas_completadas['Amenities'].str.contains('internet|ethernet', case=False), 'Internet'] = 1

datos_properties_columnas_completadas['TV'] = 0
datos_properties_columnas_completadas.loc[datos_properties_columnas_completadas['Amenities'].str.contains('tv|TV', case=False), 'TV'] = 1

datos_properties_columnas_completadas['Private Pool'] = 0
datos_properties_columnas_completadas.loc[datos_properties_columnas_completadas['Amenities'].str.contains('private pool', case=False), 'Private Pool'] = 1

datos_properties_columnas_completadas['Ac'] = 0
datos_properties_columnas_completadas.loc[datos_properties_columnas_completadas['Amenities'].str.contains('ac', case=False), 'Ac'] = 1

datos_properties_columnas_completadas

ASIGNAR A LOS NAN DE CREATED DATE LA MINIMA FECHA QUE FIGURAN EN EL DIARIO

In [ ]:
alojamientos_sin_created_date = datos_properties_columnas_completadas[datos_properties_columnas_completadas['Created Date'].isna()]
alojamientos_sin_created_date

In [ ]:
datos_properties_columnas_completadas.columns
columnas_df = pd.DataFrame(datos_properties_columnas_completadas.columns, columns=['COLUMNAS PROPERTY'])
print(columnas_df)

In [ ]:
datosProperties = datos_properties_columnas_completadas

# MERGE DATOS PROPERTY

In [ ]:
datosProperties
#datosMonthly['Reporting Month'] = datosMonthly['Reporting Month'].astype(str)
datosMonthlyReset = datosMonthly.groupby('Property ID').sum().reset_index()
datosDailyReset = datosDaily.groupby('Property ID').sum().reset_index()

datos_properties_monthly = pd.merge(datosProperties, datosMonthlyReset, on="Property ID", how="inner")
datos_properties_monthly
datos_properties_monthly_daily = pd.merge(datos_properties_monthly, datosDailyReset, on="Property ID", how="inner")
datos_properties_monthly_daily

In [ ]:
existen_duplicados = datos_properties_monthly_daily.duplicated().any()
existen_duplicados

In [ ]:
datosAlojamientos = datos_properties_monthly_daily['Property ID']
datosAlojamientos

DATOS PROPERTY

In [ ]:
datosPropertyMerge = pd.merge(datosAlojamientos, datosProperties, on="Property ID", how="inner")
datosPropertyMerge

In [ ]:
datosMonthlyMerge = pd.merge(datosAlojamientos, datosMonthly, on="Property ID", how="inner")
datosMonthlyMerge.groupby("Property ID").sum()

In [ ]:
datosDailyMerge = pd.merge(datosAlojamientos, datosDaily, on="Property ID", how="inner")
datosDailyMerge.groupby("Property ID").sum()

In [ ]:
datos_properties_columnas_completadas_merge = pd.merge(datos_properties_columnas_completadas, datosAlojamientos, on='Property ID', how='inner')
valores_ausentes_property_columnas_completadas = datos_properties_columnas_completadas_merge.isna().sum().reset_index()
valores_ausentes_property_columnas_completadas.columns = ['Columna', 'Valores Ausentes']
valores_ausentes_property_columnas_completadas = valores_ausentes_property_columnas_completadas[valores_ausentes_property_columnas_completadas['Valores Ausentes'] > 0]
valores_ausentes_property_columnas_completadas = valores_ausentes_property_columnas_completadas.sort_values(by='Valores Ausentes', ascending=False)
plt.figure(figsize=(14, 7))
sns.barplot(y='Valores Ausentes', x='Columna', data=valores_ausentes_property_columnas_completadas)
plt.title('Valores ausentes en dataset property')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/valores_ausentes_property.pdf', dpi=300, format='pdf')
print(valores_ausentes_property_columnas_completadas)

In [ ]:
filas_con_nulos_creado = datos_properties_columnas_completadas['Created Date'].isnull().sum()
filas_con_nulos_creado

# ANÁLSISI DE LAS COLUMNAS DE LOS ALOJAMIENTOS PRESENTES EN TODOS LOS DATASETS

In [ ]:
datos_properties_precio_medio = pd.merge(datosProperties, solo_precio, on="Property ID", how="inner")
datos_properties_precio_medio

In [ ]:
datos_completos_properties_finales = pd.merge(datosProperties, datosAlojamientos, on="Property ID", how="inner")
limite = 10
datos_completos_properties_finales['Max Guests Group'] = datos_completos_properties_finales['Max Guests'].apply(lambda x: '> ' + str(limite) if x > limite else str(x))

max_guests = datos_completos_properties_finales['Max Guests Group']
plt.figure(figsize=(14,7))
sns.countplot(x='Max Guests Group', data=datos_completos_properties_finales, order=datos_completos_properties_finales['Max Guests Group'].value_counts().index)
plt.xlabel('Máximo de personas')
plt.ylabel('Número de alojamientos')
plt.title('Máximo de personas por alojamiento', fontsize = 20, pad=10)
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

orden_max_guests = [str(i) for i in range(1, 11)] + ['> 10']

plt.figure(figsize=(14, 7))
sns.countplot(x='Max Guests Group', data=datos_completos_properties_finales, order=orden_max_guests)
plt.xlabel('Máximo de personas')
plt.ylabel('Frecuencia')
plt.title('Máximo de personas por alojamiento', fontsize=20, pad=10)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/max_guests.pdf', dpi=300, format='pdf')

In [ ]:
datos_completos_properties_finales[ datos_completos_properties_finales['Max Guests'] == 4]

MEDIA DE MÁXIMO DE HUÉSPEDES POR ALOJAMIENTO

In [ ]:
datos_completos_properties_finales['Max Guests'].describe()

In [ ]:
limite = 12
datos_completos_properties_finales['Max Guests Group'] = datos_completos_properties_finales['Max Guests'].apply(lambda x: '> ' + str(limite) if x > limite else str(x))
datos_completos_properties_finales[datos_completos_properties_finales['Max Guests'] < 20][['Max Guests', 'Max Guests Group']].value_counts()

In [ ]:
datos_completos_properties_finales_sin14 = datos_completos_properties_finales[datos_completos_properties_finales['Bedrooms'] != 14]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
orden_max_guests = [str(i) for i in range(1, 8)]
sns.countplot(x='Bedrooms', data=datos_completos_properties_finales_sin14, order=orden_max_guests, palette={'#5c7fb9'})


plt.xlabel('Habitaciones')
plt.ylabel('Frecuencia')
plt.gca().yaxis.grid(True, linestyle='-', linewidth=0.5, color='grey')
plt.title('Número de habitaciones por alojamiento', fontsize = 20, pad = 10)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/habitaciones_por_alojamiento.pdf', dpi=300, format='pdf')

NÚMERO MEDIO DE HABITACIONES POR ALOJAMIENTO

In [ ]:
datos_completos_properties_finales[datos_completos_properties_finales['Bedrooms'] > 0]['Bedrooms'].describe()

In [ ]:
datos_completos_properties_sin_habitacion = datos_completos_properties_finales[datos_completos_properties_finales['Bedrooms'] == 5]
datos_completos_properties_sin_habitacion[['Property ID', 'Listing Type', 'Property Type', 'Bedrooms', 'Bathrooms']]

In [ ]:
plt.figure(figsize=(14, 7))
datos_completos_properties_sin_baños_0 = datos_completos_properties_finales

limite = 3
datos_completos_properties_sin_baños_0['Max Baños'] = datos_completos_properties_sin_baños_0['Bathrooms'].apply(lambda x: '> ' + str(limite) if x > limite else str(x))

orden_max_guests = ['0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '> 3']
sns.countplot(x='Max Baños', data=datos_completos_properties_sin_baños_0, order=orden_max_guests)
plt.xlabel('Baños')
plt.ylabel('Frecuencia')
plt.title('Número de baños por alojamiento', fontsize = 20, pad = 10)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/baños_por_alojamiento.pdf', dpi=300, format='pdf')

In [ ]:
datos_completos_properties_sin_baños_0['Bathrooms'].value_counts()

In [ ]:
datos_completos_properties_sin_habitacion = datos_completos_properties_finales[datos_completos_properties_finales['Bathrooms'] == 5]
datos_completos_properties_sin_habitacion[['Property ID', 'Listing Type', 'Property Type', 'Bedrooms', 'Bathrooms']]

NÚMERO MEDIO DE BAÑOS POR ALOJAMIENTO

In [ ]:
datos_completos_properties_finales[datos_completos_properties_finales['Bathrooms'] > 0]['Bathrooms'].describe()

In [ ]:
datos_completos_properties_finales[datos_completos_properties_finales['Bathrooms'] == 0][['Property Type','Bathrooms']]

In [ ]:
plt.figure(figsize=(14,7))
sns.countplot(x='Listing Type', data=datos_completos_properties_finales)
plt.xlabel('Tipo de propiedad')
plt.ylabel('Frecuencia')
plt.title('Cantidad de alojamientos por tipo de propiedad', fontsize = 20, pad = 10)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/tipos_de_propiedad.pdf', dpi=300, format='pdf')

In [ ]:
text = ' '.join(datos_completos_properties_finales['Property Type'].astype(str))
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(14, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de palabras tipo de propiedad', fontsize = 20, pad=20)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/wordlcloud_property_type.pdf', dpi=300, format='pdf')

In [ ]:
datos_completos_properties_finales['Property Type'].value_counts()

In [ ]:
text = ' '.join(datos_completos_properties_finales['Amenities'].astype(str))
wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=50).generate(text)

plt.figure(figsize=(14, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Nube de palabras comunes en servicios por alojamiento', fontsize = 20, pad=20)
plt.axis('off')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/wordlcloud_servicios_comunes.pdf', dpi=300, format='pdf')

word_list = list(wordcloud.words_.keys())
unique_words = set(word_list)

print("Cantidad de palabras diferentes:", len(unique_words))

DESGLOSAMOS AMENITIES EN TODAS LAS COLUMNAS POSIBLES

In [ ]:
datos_properties_transformados2 = datos_completos_properties_finales

amenities_unicos = set()
for amenities_lista in datos_properties_transformados2['Amenities']:
    amenities_unicos.update(amenities_lista.split(','))

amenities_unicos

for amenitie in amenities_unicos:
    datos_properties_transformados2[amenitie.strip()] = 0

for index, row in datos_properties_transformados2.iterrows():
    for amenitie in row['Amenities'].split(','):
        datos_properties_transformados2.at[index, amenitie.strip()] = 1

datos_properties_transformados2

In [ ]:
columnas_int = datos_properties_transformados2.select_dtypes(include='int')
frecuencia_amenities = columnas_int.sum()
total_alojamientos = len(datos_properties_transformados2)

porcentaje_amenities = (frecuencia_amenities / total_alojamientos) * 100

porcentaje_amenities

amenities_mayor_50 = porcentaje_amenities[(porcentaje_amenities > 1) & (porcentaje_amenities < 10)]
amenities_mayor_50


(PRECIO MEDIO POR NOCHE)

In [ ]:
datos_diarios_reservadosss = pd.merge(datosDaily, datosProperties, on="Property ID", how="inner")
datos_diarios_reservadosss

In [ ]:
datos_diarios_reservadosss['Price (USD)'].isna().sum()
datos_diarios_reservadosss

In [ ]:
datos_diarios_reservadosss = datos_diarios_reservadosss.loc[:,~datos_diarios_reservadosss.columns.duplicated()]
datos_diarios_reservados_notna = datos_diarios_reservadosss[datos_diarios_reservadosss['Price (Native)'] != 0]

In [ ]:
#uso de datos_diarios_reservados
pd.set_option('display.float_format', lambda x: '%.2f' % x)
datos_diarios_reservados_notna['Price (Native)'].describe()

In [ ]:
plt.figure(figsize=(16,8))
sns.set_theme(style="whitegrid")
plt.xlim(0, 1935)
plt.xticks(range(0, 1935, 25))
labels = plt.xticks()[0]
new_labels = ['' if i % 50 != 0 else str(i) for i in labels]

plt.xticks(labels, new_labels)
sns.boxenplot(x='Price (USD)', data=datos_diarios_reservados_notna, width=0.9)
plt.xlabel('Precio (USD)')
plt.xticks(rotation=45)
plt.title('Diagrama de caja de precios por noche de los alojamientos',fontsize = 20, pad=20)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/boxenplot_precio_por_noche.pdf', dpi=300, format='pdf')


(EVOLUCIÓN DEL PRECIO POR AÑO, TEMPORADA, MES ...)

In [ ]:
#NECESARIO AÑADIR COLUMNAS DE MESES Y AÑOS PARA AGRUPAR, Y TAMBIÉN OTRA DE TEMPORADA

In [ ]:
#VAMOS A MOSTRAR EL PRECIO MEDIO POR NOCHE POR CADA UNO DE LOS ALOJAMIENTOS Y VERLO GRÁFICAMENTE
datos_diarios_reservados_notna['Date'] = pd.to_datetime(datos_diarios_reservados_notna['Date'])
datos_diarios_reservados_notna['Mes'] = datos_diarios_reservados_notna['Date'].dt.strftime('%Y' + '-' '%m')
datos_diarios_reservados_notna['Año'] = datos_diarios_reservados_notna['Date'].dt.strftime('%Y')

In [ ]:
datos_diarios_reservados_notna['Recuento'] = 1
datos_diarios_reservados_notna

In [ ]:
datos_diarios_reservados_notna['Año'] = datos_diarios_reservados_notna['Año'].astype(int)
datos_diarios_reservados_notna_utiles = datos_diarios_reservados_notna[['Año', 'Price (Native)', 'Recuento']]
datos_diarios_reservados_años = datos_diarios_reservados_notna_utiles.groupby('Año').sum()
datos_diarios_reservados_notna['Año'].dtype

In [ ]:
datos_diarios_reservados_años['Precio Medio'] = datos_diarios_reservados_años['Price (Native)']/datos_diarios_reservados_años['Recuento']
datos_diarios_reservados_años

TASA DE CRECIMIENTO DE LOS PRECIOS

In [ ]:
datos_diarios_reservados_años['Tasa Crecimiento'] = datos_diarios_reservados_años['Precio Medio'].pct_change() * 100
datos_diarios_reservados_años

In [ ]:
datos_diarios_reservados_años['Destacar'] = datos_diarios_reservados_años['Tasa Crecimiento'] < 0
plt.figure(figsize=(20,7))
sns.catplot(y='Tasa Crecimiento', x='Año', data=datos_diarios_reservados_años, kind='bar', hue='Destacar', palette={False: 'green', True: 'red'})
plt.title('Tasa de crecimiento del precio medio', fontsize = 20, pad = 10)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/tasa_crecimiento_precio_medio_por_noche_anual.pdf', dpi=300, format='pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,4))
sns.catplot(y='Precio Medio', x='Año', data=datos_diarios_reservados_años, kind='bar')
plt.title('Precio medio por noche', fontsize = 20, pad = 10)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/precio_medio_por_noche_anual.pdf', format='pdf', bbox_inches='tight')

In [ ]:
datos_diarios_reservados_meses = datos_diarios_reservados_notna[['Mes', 'Price (Native)', 'Recuento']].groupby('Mes').sum()
datos_diarios_reservados_meses['Precio Medio'] = datos_diarios_reservados_meses['Price (Native)']/datos_diarios_reservados_meses['Recuento']
datos_diarios_reservados_meses

In [ ]:
plt.figure(figsize=(14,7))
sns.catplot(y='Precio Medio', x='Mes', data=datos_diarios_reservados_meses, kind='bar', aspect = 4)
plt.xticks(rotation=-45)
plt.title('Precio medio por noche', fontsize = 20, pad = 10)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/precio_medio_por_noche_mensual.pdf', dpi=300, format='pdf')

PRECIO MEDIO POR NOCHE DE ALOJAMIENTO

In [ ]:
datos_diarios_reservados_unicos_conteo_final = pd.merge(datos_properties_monthly_daily, datos_diarios_reservados_unicos_conteo, on="Property ID", how="inner")
datos_diarios_reservados_unicos_conteo_final[['Property ID', 'Precio Medio']]

In [ ]:
datos_diarios_reservados_unicos_conteo_final.columns = datos_diarios_reservados_unicos_conteo_final.columns.str.replace('_x', '')
datos_diarios_reservados_unicos_conteo_final

In [ ]:
datos_diarios_reservados_unicos_conteo_final[['Property ID', 'Precio Medio']]
precio_medio_igual_a_cero = (datos_diarios_reservados_unicos_conteo_final['Precio Medio'] == 0).sum()
precio_medio_igual_a_cero

In [ ]:
rangos = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475, 50000]
etiquetas = ['0-25','26-50','51-75','76-101','101-125','126-150','151-175','176-200','201-225','226-250','251-275','276-300','301-325','326-350','351-375','376-400','401-425','426-450','451-475','476-5000']
datos_diarios_reservados_unicos_conteo_final['rango_precio'] = pd.cut(datos_diarios_reservados_unicos_conteo_final['Precio Medio'], bins=rangos, labels=etiquetas, right=False)
plt.figure(figsize=(14, 10))
sns.histplot(data=datos_diarios_reservados_unicos_conteo_final, x='rango_precio', discrete=True, kde=True, bins=[x for x in range(0,5000)])
plt.title('Distribución del precio medio por noche', fontsize = 16, pad = 10)
plt.xlabel('Rango de precios por noche')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/distribucion_precio_medio_apartamento.pdf', dpi=300, format='pdf')

In [ ]:
datos_diarios_reservados_unicos_conteo_final['rango_precio'].value_counts().sum()

In [ ]:
#AÑADIR VALORACIONES
#AÑADIR VALORACIONES DE LIMPIEZA
datos_diarios_reservados_unicos_conteo_final.columns
precio_medio_por_piscina = datos_diarios_reservados_unicos_conteo_final.groupby('Pool')['Precio Medio'].mean()
precio_medio_por_piscina

In [ ]:
ultimo_año = pd.read_csv('/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/df_ultimos_12_meses_revenue.csv')
precio_medio_por_piscina_ultimo_año = ultimo_año.groupby('Pool')['ADR (Native)'].mean()
precio_medio_por_piscina_ultimo_año

In [ ]:
datos_completos_properties_creados = datos_completos_properties_finales['Created Date']
datos_completos_properties_creados

PORCENTAJE DE ALOJAMIENTOS CON PISCINA

In [ ]:
datos_completos_properties_finales[datos_completos_properties_finales['Amenities'].str.contains('pool|swim', case=False)].shape[0]

In [ ]:
proporcion_pool = datos_completos_properties_finales[datos_completos_properties_finales['Amenities'].str.contains('pool|swim|swimming', case=False)].shape[0] / 13358
proporcion_sin_pool = 1 - proporcion_pool

sizes = [proporcion_pool, proporcion_sin_pool]
labels = ['Con piscina', 'Sin piscina']
colors = ['#01881C', '#BE2414']

plt.figure(figsize=(10, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140, wedgeprops=dict(width=0.3), textprops={'fontsize': 12})
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.axis('equal')

plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/porcentaje_alojamientos_piscina.pdf', dpi=300, format='pdf')

plt.show()

In [ ]:
datos_completos_properties_finales[datos_completos_properties_finales['Amenities'].str.contains('free_parking', case=False)].shape[0]

In [ ]:
proporcion_parking = datos_completos_properties_finales[datos_completos_properties_finales['Amenities'].str.contains('free_parking', case=False)].shape[0] / 13358
proporcion_sin_parking = 1 - proporcion_parking

sizes = [proporcion_parking, proporcion_sin_parking]
labels = ['Con parking', 'Sin parking']
colors = ['#01881C', '#BE2414']

plt.figure(figsize=(10, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140, wedgeprops=dict(width=0.3), textprops={'fontsize': 12})

centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.axis('equal')

plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/porcentaje_alojamientos_parking.pdf', dpi=300, format='pdf')

plt.show()

PORCENTAJE DE ALOJAMIENTOS CON ASCENSOR

In [ ]:
datos_completos_properties_finales[datos_completos_properties_finales['Amenities'].str.contains('elevator', case=False)].shape[0]

In [ ]:
proporcion_elevator = datos_completos_properties_finales[datos_completos_properties_finales['Amenities'].str.contains('elevator', case=False)].shape[0] / 13358
proporcion_sin_elevator = 1 - proporcion_elevator

sizes = [proporcion_elevator, proporcion_sin_elevator]
labels = ['Con ascensor', 'Sin ascensor']
colors = ['#01881C', '#BE2414']

plt.figure(figsize=(10, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140, wedgeprops=dict(width=0.3), textprops={'fontsize': 12})

centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.axis('equal')

plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/porcentaje_alojamientos_ascensor.pdf', dpi=300, format='pdf')

plt.show()

In [ ]:
datos_completos_properties_finales[datos_completos_properties_finales['Amenities'].str.contains('private pool', case=False)].shape[0]

In [ ]:
proporcion_aire = datos_completos_properties_finales[datos_completos_properties_finales['Amenities'].str.contains('air', case=False)].shape[0]
proporcion_aire

EVOLUCIÓN DE LA INCORPORACIÓN DE LOS APARTAMENTOS A LAS PLATAFORMAS

In [ ]:
datos_completos_properties_finales[datos_completos_properties_finales['Created Date'].isnull()]

EVOLUCIÓN DE NUEVOS ALOJAMIENTOS DADOS DE ALTA EN LAS PLATAFORMAS

In [ ]:
datos_completos_properties_finales['Created Date'] = pd.to_datetime(datos_completos_properties_finales['Created Date'])
datos_completos_properties_finales['Año Creacion'] = datos_completos_properties_finales['Created Date'].dt.strftime('%Y')

filtro = datos_completos_properties_finales[datos_completos_properties_finales['Año Creacion'].notnull()]

filtro['Recuento'] = 1
evolucion_nuevos_alojamientos = filtro[['Año Creacion', 'Recuento']].groupby('Año Creacion').sum()

In [ ]:
evolucion_nuevos_alojamientos

PROCESO DE SACAR % DEL TOTAL CADA UNO DE LOS AÑOS PARA COMPLETAR LOS QUE NO TIENEN FECHA DE CREACIÓN

In [ ]:
totales = evolucion_nuevos_alojamientos['Recuento'].sum()
evolucion_nuevos_alojamientos['Porcentaje'] = (evolucion_nuevos_alojamientos['Recuento'] / totales)
evolucion_nuevos_alojamientos['Recuento'] = evolucion_nuevos_alojamientos['Recuento']
evolucion_nuevos_alojamientos['Recuento'].sum()

In [ ]:
plt.figure(figsize=(14, 8))
sns.catplot(y='Recuento', x='Año Creacion', data=evolucion_nuevos_alojamientos, kind='bar', height=6, aspect=2)
plt.title('Nuevos alojamientos en las plataformas', fontsize=20, pad=20)
plt.xlabel('Año de creación', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', linewidth=0.5, color='gray')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/nuevos_alojamientos.pdf', format='pdf', dpi=300)

In [ ]:
evolucion_nuevos_alojamientos['Recuento'].sum()

In [ ]:
evolucion_nuevos_alojamientos['Acumulado'] = evolucion_nuevos_alojamientos['Recuento'].cumsum()
evolucion_nuevos_alojamientos

# ANÁLISIS DE LA OFERTA

In [ ]:
datosAlojamientos

In [ ]:
datos_oferta = datos_diarios[(datos_diarios['Status'] == 'R') | (datos_diarios['Status'] == 'A')]
datos_oferta

In [ ]:
datos_oferta_alojamientos = pd.merge(datos_oferta, datosAlojamientos, on='Property ID', how='inner')
datos_oferta_alojamientos

In [ ]:
datos_oferta_alojamientos['Date'] = pd.to_datetime(datos_oferta_alojamientos['Date'], format='%Y-%m-%d')
datos_oferta_alojamientos['Año'] = datos_oferta_alojamientos['Date'].dt.year
datos_oferta_alojamientos

In [ ]:
datos_oferta_alojamientos2 = datos_oferta_alojamientos.groupby(['Año', 'Property ID']).size().reset_index(name='count')
datos_oferta_alojamientos2['Count 2'] = 1
datos_oferta_alojamientos2.groupby('Año').sum()

In [ ]:
datos_oferta_alojamientos3 = datos_oferta_alojamientos.groupby('Año')['Property ID'].nunique().reset_index()
datos_oferta_alojamientos3.columns = ['Año', 'Cantidad de Properties']
datos_oferta_alojamientos3

In [ ]:
plt.figure(figsize=(14, 7))
sns.barplot(y='Cantidad de Properties', x='Año',data=datos_oferta_alojamientos3)
plt.title("Alojamientos ofertados por año")
plt.ylabel("Frecuencia")
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/alojamientos_ofertados_año.pdf', dpi=600, format='pdf')

In [ ]:
datos_diarios_merge = pd.merge(datos_diarios, datosAlojamientos, on='Property ID', how='inner')
datos_diarios_merge_a_r = datos_diarios_merge[(datos_diarios_merge['Status'] == 'A') | (datos_diarios_merge['Status'] == 'R') | (datos_diarios_merge['Status'] == 'B')]
datos_diarios_merge_a_r['Date'] = pd.to_datetime(datos_diarios_merge_a_r['Date'], format='%Y-%m-%d')
datos_diarios_merge_a_r['Año'] = datos_diarios_merge_a_r['Date'].dt.year
datos_diarios_merge_a_r.groupby('Año')['Property ID'].nunique()

ANÁLISIS DE LA DEMANDA, OFERTADOS / RESERVADOS

In [ ]:
datos_oferta_alojamientos_available = datos_oferta_alojamientos[datos_oferta_alojamientos['Status'] == 'A'].groupby('Año').size().reset_index()
datos_oferta_alojamientos_available.columns = ['Año', 'Dias Disponibles']
datos_oferta_alojamientos_available

In [ ]:
datos_oferta_alojamientos222 = pd.merge(datos_oferta, datosAlojamientos, on='Property ID', how='inner')
datos_oferta_alojamientos222[(datos_oferta_alojamientos222['Status'] == 'A') | (datos_oferta_alojamientos222['Status'] == 'R')].groupby('Property ID').sum()

In [ ]:
datos_oferta_alojamientos_reserved = datos_oferta_alojamientos[datos_oferta_alojamientos['Status'] == 'R'].groupby('Año').size().reset_index()
datos_oferta_alojamientos_reserved.columns = ['Año', 'Dias Reservados']
datos_oferta_alojamientos_reserved

In [ ]:
datos_oferta_alojamientos_ocupacion = pd.DataFrame()
datos_oferta_alojamientos_ocupacion['Año'] = datos_oferta_alojamientos_reserved['Año']
datos_oferta_alojamientos_ocupacion['Tasa de Ocupación'] = datos_oferta_alojamientos_reserved['Dias Reservados'] / (datos_oferta_alojamientos_reserved['Dias Reservados'] + datos_oferta_alojamientos_available['Dias Disponibles'])
datos_oferta_alojamientos_ocupacion

In [ ]:
datos_verano = datos_oferta_alojamientos[(datos_oferta_alojamientos['Date'].dt.month > 7) & (datos_oferta_alojamientos['Date'].dt.month < 9)]
datos_oferta_alojamientos_available_verano = datos_verano[datos_verano['Status'] == 'A'].groupby('Año').size().reset_index()
datos_oferta_alojamientos_available_verano.columns = ['Año', 'Dias Disponibles Verano']
datos_oferta_alojamientos_available_verano

In [ ]:
datos_oferta_alojamientos_reserved_verano = datos_verano[datos_verano['Status'] == 'R'].groupby('Año').size().reset_index()
datos_oferta_alojamientos_reserved_verano.columns = ['Año', 'Dias Reservados Verano']
datos_oferta_alojamientos_reserved_verano

In [ ]:
datos_oferta_alojamientos_ocupacion_verano = pd.DataFrame()
datos_oferta_alojamientos_ocupacion_verano['Año'] = datos_oferta_alojamientos_reserved_verano['Año']
datos_oferta_alojamientos_ocupacion_verano['Tasa de Ocupación'] = datos_oferta_alojamientos_reserved_verano['Dias Reservados Verano'] / (datos_oferta_alojamientos_reserved_verano['Dias Reservados Verano'] + datos_oferta_alojamientos_available_verano['Dias Disponibles Verano'])
datos_oferta_alojamientos_ocupacion_verano

# ANÁLISIS CORRELATIVO

DATASET PARA EL ANÁLISIS COORELATIVO

In [ ]:
datos_diarios_reservados_unicos_conteo.columns

In [ ]:
datos_diarios_reservados_unicos_conteo

In [ ]:
datos_diarios_reservados_unicos_conteo_final.columns

In [ ]:
datos_diarios_reservados_unicos_conteo_final[['Reservation Days', 'Available Days' ]]

In [ ]:
datos_diarios_reservados_unicos_conteo_final['Ocupacion'] = datos_diarios_reservados_unicos_conteo_final['Reservation Days'] / (datos_diarios_reservados_unicos_conteo_final['Reservation Days'] + datos_diarios_reservados_unicos_conteo_final['Available Days'])

In [ ]:
datos_diarios_reservados_unicos_conteo_final['Number of Reservations']

In [ ]:
datos_properties_reservados_corr = datos_diarios_reservados_unicos_conteo_final[['Property ID', 'Precio Medio']]
datos_properties_columnas_corr = datos_diarios_reservados_unicos_conteo_final[['Property ID', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Pool', 'Lift/Elevator', 'Parking', 'Internet', 'TV', 'Ocupacion', 'Number of Reservations', 'Number of Photos', 'Private Pool']]
datos_properties_corr = pd.merge(datos_properties_reservados_corr, datos_properties_columnas_corr, how='inner')

In [ ]:
ax = sns.heatmap(
    datos_properties_corr[['Precio Medio', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Pool', 'Lift/Elevator', 'Parking', 'Internet', 'TV', 'Ocupacion', 'Number of Reservations', 'Number of Photos', 'Private Pool']].corr(),
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

ax.grid(False, 'major')
ax.grid(True, 'minor')
ax.set_xticks([t + 0.5 for t in ax.get_xticks()], minor=True)
ax.set_yticks([t + 0.5 for t in ax.get_yticks()], minor=True)

In [ ]:
def heatmap(x, y, size):
    fig, ax = plt.subplots()

    x_labels = [v for v in sorted(x.unique())]
    y_labels = [v for v in sorted(y.unique())]
    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)}
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)}

    size_scale = 500
    ax.scatter(
        x=x.map(x_to_num),
        y=y.map(y_to_num),
        s=size * size_scale,
        marker='s'
    )
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, rotation=45, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels)

In [ ]:
datos_properties_reservados_corr2 = datos_diarios_reservados_unicos_conteo_final[['Property ID', 'Precio Medio']]
datos_properties_columnas_corr2 = datos_diarios_reservados_unicos_conteo_final[['Property ID', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Pool', 'Lift/Elevator', 'Parking', 'Internet', 'TV', 'Ocupacion', 'Number of Reservations', 'Number of Photos', 'Private Pool']]
datos_properties_corr2 = pd.merge(datos_properties_reservados_corr2, datos_properties_columnas_corr2, how='inner')
columnas_relevantes = datos_properties_corr2[['Precio Medio', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Pool', 'Lift/Elevator', 'Parking', 'Internet', 'TV', 'Ocupacion', 'Number of Reservations', 'Number of Photos', 'Private Pool']]
correlacion = columnas_relevantes.corr()
correlacion

In [ ]:
coef = columnas_relevantes.corr(method='pearson')
coef

In [ ]:
plt.figure(figsize=(14,6))
sns.heatmap(coef, annot=True)
plt.show()

# GRÁGICAS POR MESES

In [ ]:
#CON ESTO TENEMOS EN DATOS_MESES TODOS LOS DATOS DE LOS APARTAMENTOS POR MESES QUE APARECEN EN MONTHLY Y DAILY (INNER)
#datos_meses2 = pd.merge(datosMonthly, datosAlojamientos, on='Property ID', how='inner')
datos_meses2 = pd.merge(datos_meses_columnas, datosAlojamientos, on='Property ID', how='inner')
datos_meses2

In [ ]:
datos_meses2.groupby('Property ID').sum()

In [ ]:
datos_meses2['Reporting Month']=pd.to_datetime(datos_meses2['Reporting Month'])
datos_meses2['Year'] = datos_meses2['Reporting Month'].dt.year
datos_meses2

In [ ]:
datos = pd.merge(datos_meses, datosAlojamientos, on='Property ID', how='inner')
datos['Reporting Month']=pd.to_datetime(datos['Reporting Month'])
datos['Year'] = datos['Reporting Month'].dt.year
datos_meses2.groupby('Year')['Property ID'].nunique()

In [ ]:
reservas_año = datos_meses2.groupby('Year')['Number of Reservations'].sum()
reservas_año

In [ ]:
reservas_año_columnas = pd.DataFrame(reservas_año)
reservas_año_columnas_reseteado = reservas_año_columnas.reset_index()

In [ ]:
#RESERVAS POR MESES CADA UNO DE LOS AÑOS (MESES QUE CADA UNO DE LOS APARTAMENTOS HAN SIDO ALQUILADOS)
#ESTAN EN datos_meses_alquilados
reservas_totales_año = pd.DataFrame(reservas_año, columns=[ 'Number of Reservations'])
plt.figure(figsize=(20,7))

unique_years = reservas_año_columnas_reseteado['Year'].nunique()
palette = sns.color_palette("husl", unique_years)

sns.barplot(x='Year', y='Number of Reservations', data=reservas_año_columnas_reseteado, palette=palette)
plt.title('Número total de reservas por año', fontsize = 20, pad = 10)
plt.xticks(rotation=-45)
plt.xlabel('Año')
plt.ylabel('Número de reservas')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/numero_reservas_por_año.pdf', dpi=300, format='pdf')

In [ ]:
datos_meses2_verano = datos_meses2
datos_meses2_verano = datos_meses2_verano[(datos_meses2_verano['Reporting Month'].dt.month >= 6) & (datos_meses2_verano['Reporting Month'].dt.month <= 8)]
reservas_verano = datos_meses2_verano.groupby('Year')['Number of Reservations'].sum()

reservas_año_verano = pd.DataFrame(reservas_verano)
new_row = pd.DataFrame({'Year': [2016], 'Reservas': [0]})
reservas_año_verano = pd.concat([new_row, reservas_año_verano], ignore_index=True)
reservas_año_columnas_resetead2 = reservas_año_verano.reset_index()
reservas_año_columnas_reseteado['Number of Reservations Verano'] = reservas_año_columnas_resetead2['Number of Reservations']
reservas_año_columnas_reseteado

In [ ]:
reservas_año_verano

In [ ]:
reservas_año_columnas_reseteado['Porcentaje Verano'] = (reservas_año_columnas_reseteado['Number of Reservations Verano'] / reservas_año_columnas_reseteado['Number of Reservations'])*100
reservas_año_columnas_reseteado

In [ ]:
import numpy as np
años = reservas_año_columnas_reseteado['Year']
reservas_anual = reservas_año_columnas_reseteado['Number of Reservations']
reservas_verano = reservas_año_columnas_reseteado['Number of Reservations Verano']
percent_verano = reservas_año_columnas_reseteado['Porcentaje Verano']

bar_width = 0.35
r1 = np.arange(len(años))
r2 = [x + bar_width for x in r1]
plt.figure(figsize=(16, 7))

plt.bar(r1, reservas_anual, color='b', width=bar_width, edgecolor='grey', label='Reservas anuales')
plt.bar(r2, reservas_verano, color='#EB984E', width=bar_width, edgecolor='grey', label='Reservas junio a agosto')

plt.xlabel('Año', fontweight='bold', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.title('Número de reservas', fontsize=16)
plt.xticks([r + bar_width / 2 for r in range(len(años))], años, fontsize=12)

fig = plt.gcf()
fig.subplots_adjust(right=0.75)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

for i, perc in enumerate(percent_verano):
    plt.text(r2[i], reservas_verano[i] + 50, f'{perc:.1f}%', ha='center', va='bottom', fontsize=10)
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/numero_reservas.pdf', dpi=1000, format='pdf')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

años = reservas_año_columnas_reseteado['Year']
reservas_anual = reservas_año_columnas_reseteado['Number of Reservations']
reservas_verano = reservas_año_columnas_reseteado['Number of Reservations Verano']
percent_verano = reservas_año_columnas_reseteado['Porcentaje Verano']

plt.figure(figsize=(16, 7))

plt.plot(años, reservas_anual, color='b', marker='o', linestyle='-', label='Reservas anuales')
plt.plot(años, reservas_verano, color='#EB984E', marker='o', linestyle='-', label='Reservas junio a septiembre')

plt.xlabel('Año', fontweight='bold', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.title('Número de reservas', fontsize=16)
plt.xticks(años, fontsize=12)

fig = plt.gcf()
fig.subplots_adjust(right=0.75)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Añadir porcentajes sobre las barras de "Reservas junio a septiembre"
for i, perc in enumerate(percent_verano):
    plt.text(años[i], reservas_verano[i] + 50, f'{perc:.1f}%', ha='center', va='bottom', fontsize=10)

# Mostrar el gráfico
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/numero_reservas_lineal.pdf', dpi=1000, format='pdf')
plt.show()

In [ ]:
reservas_totales_año

In [ ]:
apartamentos_reservados_por_año = datos_meses2.groupby('Year')['Property ID'].nunique()
apartamentos_reservados_por_año.reset_index(name='Alojamientos Diferentes')

In [ ]:
plt.figure(figsize=(20, 7))
sns.barplot(x='Year',y='Alojamientos Diferentes', data=apartamentos_reservados_por_año.reset_index(name='Alojamientos Diferentes'), hue='Year', palette='viridis')
plt.legend(bbox_to_anchor=(1.01, 1.01), loc=2)
plt.title('Número total de apartamentos reservados por año', fontsize = 20, pad = 10)
plt.xticks(rotation=-45)
plt.xlabel('Año')
plt.ylabel('Apartamentos')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/numero_apartamentos_reservados_por_año.pdf', dpi=300, format='pdf')

In [ ]:
#datos_meses2['conteo']=1
datos_meses_ocupacion = datos_meses2.groupby('Reporting Month').sum().reset_index()

datos_meses_ocupacion['Dias Ofertados'] = datos_meses_ocupacion['Reservation Days'] + datos_meses_ocupacion['Available Days']
datos_meses_ocupacion['Porcentaje Ocupacion'] = datos_meses_ocupacion['Reservation Days'] / datos_meses_ocupacion['Dias Ofertados']
datos_meses_ocupacion

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(datos_meses_ocupacion['Reporting Month'], datos_meses_ocupacion['Porcentaje Ocupacion'], marker='o')

plt.xlabel('Meses')
plt.ylabel('Porcentaje de Ocupación')
plt.title('Evolución de la Tasa de Ocupación')

plt.xticks(rotation=45)

plt.grid(True)
plt.tight_layout()
plt.show()
#faltaría el plt.savefig

NUMERO TOTAL DE RESERVAS POR MESES


In [ ]:

#filtro de solo los que en cada uno de los meses han sido alquilados
#datos_meses2 = datos_meses2[datos_meses2['Occupancy Rate'] > 0]

datos_meses_agrupados_mes = datos_meses2.groupby('Reporting Month').sum().reset_index()
datos_meses_agrupados_mes

In [ ]:
plt.figure(figsize=(200, 7))
orden=datos_meses2['Reporting Month'].sort_values()
sns.catplot(x='Reporting Month', y='Number of Reservations', data=datos_meses2.groupby('Reporting Month').sum().reset_index(),kind='bar', palette='viridis', aspect=4)
plt.legend(bbox_to_anchor=(1.01, 1.01), loc=2)  # Para sacar la leyenda fuera del gráfico
plt.xticks(rotation=-45)
plt.title('Número total de rservas por meses', fontsize = 20, pad = 10)
plt.xlabel('Mes')
plt.ylabel('Número de reservas')
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/numero_reservas_por_meses.pdf', dpi=300, format='pdf')

MEDIA DE LAS ESTANCIAS POR RESERVAS DE CADA UNO DE LOS MESES

In [ ]:
#PARA ELLO NECESITAMOS REPORTING MONTH, NUMBER OF RESERVATIONS, RESERVATIONS DAY Y LA COLUMNA CALCULADA A PARTIR DE LOS 2 ANTERIORES
datos_noches_reservas = datos_meses_agrupados_mes[['Reporting Month', 'Number of Reservations', 'Reservation Days']]
datos_noches_reservas['Media Estancia'] = datos_noches_reservas['Reservation Days']/datos_noches_reservas['Number of Reservations']
datos_noches_reservas

plt.figure(figsize=(14, 7))
sns.catplot(x='Reporting Month', y='Media Estancia', data=datos_noches_reservas, kind='bar', aspect=4)
plt.xticks(rotation=-45)
plt.title('Estacia media por meses ', fontsize = 20, pad = 10)
plt.xlabel('Mes')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/media_De_las_estancias_meses.pdf', dpi=300, format='pdf')

In [ ]:
datos_noches_reservas

In [ ]:
datos_noches_reservas['Reporting Month'] = pd.to_datetime(datos_noches_reservas['Reporting Month'])
datos_noches_reservas['Año'] = datos_noches_reservas['Reporting Month'].dt.year
datos_noches_reservas_año = datos_noches_reservas.groupby('Año')['Media Estancia'].mean().reset_index()
datos_noches_reservas_año

In [ ]:
datos_noches_reservas_año_verano = datos_noches_reservas
datos_noches_reservas_año_verano = datos_noches_reservas[(datos_noches_reservas['Reporting Month'].dt.month >=6) & (datos_noches_reservas['Reporting Month'].dt.month <=8)]
datos_noches_reservas_año_verano_año = datos_noches_reservas_año_verano.groupby('Año')['Media Estancia'].mean().reset_index()
new_row = pd.DataFrame({'Year': [2016], 'Reservas': [0]})
datos_noches_reservas_año_verano_año = pd.concat([new_row, datos_noches_reservas_año_verano_año], ignore_index=True)
datos_noches_reservas_año_verano_año

In [ ]:
datos_noches_reservas_año_verano = pd.DataFrame()
datos_noches_reservas_año_verano = datos_noches_reservas_año
datos_noches_reservas_año_verano['Media Estancia Verano'] = datos_noches_reservas_año_verano_año['Media Estancia']
datos_noches_reservas_año_verano

In [ ]:
años = datos_noches_reservas_año_verano['Año']
estancia_media = datos_noches_reservas_año_verano['Media Estancia']
estancia_media_verano = datos_noches_reservas_año_verano['Media Estancia Verano']

bar_width = 0.35
r1 = np.arange(len(años))
r2 = [x + bar_width for x in r1]

plt.figure(figsize=(16, 7))

plt.bar(r1, estancia_media, color='b', width=bar_width, edgecolor='grey', label='Estancia media anual')
plt.bar(r2, estancia_media_verano, color='#EB984E', width=bar_width, edgecolor='grey', label='Estancia media junio a agosto')

plt.xlabel('Año', fontweight='bold')
plt.ylabel('Frecuencia')
plt.title('Estancia media por reserva')
plt.xticks([r + bar_width / 2 for r in range(len(años))], años)  # Ajustar etiquetas para que se centren entre las barras

fig = plt.gcf()
fig.subplots_adjust(right=0.75)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/estancia_media.pdf', dpi=1000, format='pdf')

PORCENTAJE DE OCUPACIÓN POR MES

In [ ]:
datos_ocupacion_meses = datos_meses_agrupados_mes[['Reporting Month', 'Reservation Days', 'Available Days']]
datos_ocupacion_meses['Dias Ofertados'] = datos_ocupacion_meses['Reservation Days']+datos_ocupacion_meses['Available Days']
datos_ocupacion_meses['Ocupacion'] = (datos_ocupacion_meses['Reservation Days']/datos_ocupacion_meses['Dias Ofertados'])*100

plt.figure(figsize=(14, 7))
sns.catplot(x='Reporting Month', y='Ocupacion', data=datos_ocupacion_meses, kind='bar', aspect=4)
plt.xticks(rotation=-45)
plt.title('Porcentaje de ocupación por meses', fontsize = 20, pad = 10)
plt.xlabel('Mes')
plt.ylabel('Porcentaje ocupación')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/porcentaje_ocupacion_por_meses.pdf', dpi=300, format='pdf')

In [ ]:
datos_ocupacion_meses

In [ ]:
datos_ocupacion_meses['Reporting Month'] = pd.to_datetime(datos_ocupacion_meses['Reporting Month'])
datos_ocupacion_meses['Año'] = datos_ocupacion_meses['Reporting Month'].dt.year
datos_ocupacion_meses[['Reservation Days', 'Available Days', 'Dias Ofertados', 'Año']].groupby('Año').sum()

In [ ]:
datos_meses_oferta_anual = datos_ocupacion_meses[['Reservation Days', 'Available Days', 'Dias Ofertados', 'Año']].groupby('Año').sum()
datos_meses_oferta_anual['Ocupacion'] = datos_meses_oferta_anual['Reservation Days'] / datos_meses_oferta_anual['Dias Ofertados']
datos_meses_oferta_anual

In [ ]:
datos_meses_oferta_vernao = datos_ocupacion_meses[(datos_ocupacion_meses['Reporting Month'].dt.month >= 6) & (datos_ocupacion_meses['Reporting Month'].dt.month <= 8)]
datos_meses_oferta_vernao['Reporting Month'] = pd.to_datetime(datos_meses_oferta_vernao['Reporting Month'])
datos_meses_oferta_vernao['Año'] = datos_meses_oferta_vernao['Reporting Month'].dt.year
datos_meses_oferta_vernao[['Reservation Days', 'Available Days', 'Dias Ofertados', 'Año']].groupby('Año').sum()

In [ ]:
datos_meses_oferta_vernao_anual = datos_meses_oferta_vernao[['Reservation Days', 'Available Days', 'Dias Ofertados', 'Año']].groupby('Año').sum()
calculo = datos_meses_oferta_vernao_anual['Reservation Days'] / datos_meses_oferta_vernao_anual['Dias Ofertados']
datos_meses_oferta_anual['Ocupacion Verano'] = calculo
datos_meses_oferta_anual

In [ ]:
datos_meses_oferta_vernao_anual['Reservation Days'] / datos_meses_oferta_vernao_anual['Dias Ofertados']

In [ ]:
datos_meses_oferta_anual = datos_meses_oferta_anual.reset_index()
datos_meses_oferta_anual

In [ ]:
import numpy as np

años = datos_meses_oferta_anual['Año']
ocupacion = datos_meses_oferta_anual['Ocupacion']
ocupacion_verano = datos_meses_oferta_anual['Ocupacion Verano']

bar_width = 0.25
r1 = np.arange(len(años))
r2 = [x + bar_width for x in r1]
plt.figure(figsize=(16, 7))

plt.bar(r1, ocupacion, color='b', width=bar_width, edgecolor='grey', label='Ocupación Media Anual')
plt.bar(r2, ocupacion_verano, color='#EB984E', width=bar_width, edgecolor='grey', label='Ocupación Media temporada alta')

plt.xlabel('Año', fontweight='bold')
plt.ylabel('Porcentaje de ocupación')
plt.title('Ocupación de alojamientos')
plt.xticks([r + bar_width for r in range(len(años))], años)

fig = plt.gcf()
fig.subplots_adjust(right=0.75)


plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/evolucion_ocupacion.pdf', dpi=1000, format='pdf')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

años = datos_meses_oferta_anual['Año']
ocupacion = datos_meses_oferta_anual['Ocupacion']
ocupacion_verano = datos_meses_oferta_anual['Ocupacion Verano']
plt.figure(figsize=(16, 7))

plt.plot(años, ocupacion, color='b', marker='o', linestyle='-', label='Ocupación Media Anual')
plt.plot(años, ocupacion_verano, color='#EB984E', marker='o', linestyle='-', label='Ocupación Media Verano')

plt.xlabel('Año', fontweight='bold')
plt.ylabel('Porcentaje de ocupación')
plt.title('Ocupación de alojamientos')
plt.xticks(años)

fig = plt.gcf()
fig.subplots_adjust(right=0.75)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/evolucion_ocupacion_lineal.pdf', dpi=1000, format='pdf')
plt.show()


EVOLUCIÓN DE OFERTA Y DEMANDA

In [ ]:
dat = datos_ocupacion_meses[['Reporting Month', 'Reservation Days', 'Dias Ofertados']]
df_long = dat.melt(id_vars='Reporting Month', var_name='Tipo', value_name='Dias')
sns.catplot(data=df_long, x='Reporting Month', y='Dias', hue='Tipo',kind='bar', aspect=4)
plt.xticks(rotation=-45)

In [ ]:
prueba = datos_meses2[datos_meses2['Reporting Month'] == '2016-09-01']['Number of Reservations'].sum()
prueba

TOTAL INGRESOS ANUALES

In [ ]:
ue_verano = datos_meses2.groupby('Year')['Revenue (Native)'].sum()
ue_verano2 = ue_verano.reset_index()

In [ ]:
ue = datos_meses2.groupby('Year')['Revenue (Native)'].sum()
ue2 = ue.reset_index()
ue2.columns = ['Year', 'Ingresos Totales']
ue2

nueva_fila = pd.DataFrame({'Year': [2016], 'Revenue (Native)': [0]})

ue_verano2 = pd.concat([nueva_fila, ue_verano2], ignore_index=True)
ue_verano2

ingresos_anual_verano = ue2

ingresos_anual_verano['Total Ingresos Verano'] = ue_verano2['Revenue (Native)']
ingresos_anual_verano['Porcentaje Verano'] = (ingresos_anual_verano['Total Ingresos Verano'] / ingresos_anual_verano['Ingresos Totales'])*100
ingresos_anual_verano

In [ ]:
from matplotlib.ticker import FuncFormatter

años = ingresos_anual_verano['Year']
ingresos_totales = ingresos_anual_verano['Ingresos Totales']
ingresos_totales_verano = ingresos_anual_verano['Total Ingresos Verano']
percent_verano = ingresos_anual_verano['Porcentaje Verano']

bar_width = 0.35
r1 = np.arange(len(años))
r2 = [x + bar_width for x in r1]

def millones(x, pos):
    return '%1.1fM' % (x * 1e-6)

plt.figure(figsize=(16, 7))

plt.bar(r1, ingresos_totales, color='b', width=bar_width, edgecolor='grey', label='Ingresos anuales')
plt.bar(r2, ingresos_totales_verano, color='#EB984E', width=bar_width, edgecolor='grey', label='Ingresos junio a agosto')

plt.xlabel('Año', fontweight='bold', fontsize=12)
plt.ylabel('Ingresos (en millones)', fontsize=12)
plt.title('Ingresos totales', fontsize=16)
plt.xticks([r + bar_width / 2 for r in range(len(años))], años, fontsize=12)

fig = plt.gcf()
fig.subplots_adjust(right=0.75)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

for i, perc in enumerate(percent_verano):
    plt.text(r2[i], ingresos_totales_verano[i] + 50, f'{perc:.1f}%', ha='center', va='bottom', fontsize=10)

formatter = FuncFormatter(millones)
plt.gca().yaxis.set_major_formatter(formatter)


plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/ingresos_totales.pdf', dpi=1000, format='pdf')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter

años = ingresos_anual_verano['Year']
ingresos_totales = ingresos_anual_verano['Ingresos Totales']
ingresos_totales_verano = ingresos_anual_verano['Total Ingresos Verano']
percent_verano = ingresos_anual_verano['Porcentaje Verano']

def millones(x, pos):
    return '%1.1fM' % (x * 1e-6)

plt.figure(figsize=(16, 7))

plt.plot(años, ingresos_totales, color='b', marker='o', linestyle='-', label='Ingresos anuales')
plt.plot(años, ingresos_totales_verano, color='#EB984E', marker='o', linestyle='-', label='Ingresos junio a septiembre')
plt.xlabel('Año', fontweight='bold', fontsize=12)
plt.ylabel('Ingresos (en millones)', fontsize=12)
plt.title('Ingresos totales', fontsize=16)
plt.xticks(años, fontsize=12)

formatter = FuncFormatter(millones)
plt.gca().yaxis.set_major_formatter(formatter)

fig = plt.gcf()
fig.subplots_adjust(right=0.75)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

for i, perc in enumerate(percent_verano):
    plt.text(años[i], ingresos_totales_verano[i], f'{perc:.1f}%', ha='center', va='bottom', fontsize=10)


plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/ingresos_totales_lineal.pdf', dpi=1000, format='pdf')
plt.show()


INGRESOS POR ESTANCIA

In [ ]:
datos_noches_reservas_año_verano

In [ ]:
diarios_reservados_merge = datos_diarios_reservados_notna
diarios_reservados_merge_verano = diarios_reservados_merge[(diarios_reservados_merge['Date'].dt.month >= 6) & (diarios_reservados_merge['Date'].dt.month <= 8)]
diarios_reservados_merge_verano['Año'] = diarios_reservados_merge_verano['Date'].dt.year
diarios_reservados_merge_verano

In [ ]:
precio_medio_verano = diarios_reservados_merge_verano.groupby('Año')['Price (Native)'].mean().reset_index()
precio_medio_verano

In [ ]:
precio_medio_anual = datos_diarios_reservados_notna.groupby('Año')['Price (Native)'].mean().reset_index()
precio_medio_anual

In [ ]:
precio_medio = pd.merge(precio_medio_verano, precio_medio_anual, on='Año', how='right')
precio_medio.columns = ['Año', 'Precio Medio Verano', 'Precio Medio Anual']
precio_medio

In [ ]:
años = precio_medio['Año']
precio_verano = precio_medio['Precio Medio Verano']
precio_anual = precio_medio['Precio Medio Anual']

bar_width = 0.35
r1 = np.arange(len(años))
r2 = [x + bar_width for x in r1]
plt.figure(figsize=(16, 7))

plt.bar(r1, precio_anual, color='b', width=bar_width, edgecolor='grey', label='Precio medio anual')
plt.bar(r2, precio_verano, color='#EB984E', width=bar_width, edgecolor='grey', label='Precio medio junio a agosto')

plt.xlabel('Año', fontweight='bold', fontsize=12)
plt.ylabel('Precio Medio', fontsize=12)
plt.title('Precio medio por noche', fontsize=16)
plt.xticks([r + bar_width / 2 for r in range(len(años))], años, fontsize=12)  # Ajustar etiquetas para que se centren entre las barras

fig = plt.gcf()
fig.subplots_adjust(right=0.75)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)

plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/precio_medio_anual_verano.pdf', dpi=1000, format='pdf')
plt.show()

In [ ]:
datos_noches_reservas_año_verano = datos_noches_reservas_año_verano.fillna(0)
datos_noches_reservas_año_verano

In [ ]:
precio_medio

In [ ]:
datos_noches_reservas_año_verano

In [ ]:
ingresos_por_estancia = pd.DataFrame()
ingresos_por_estancia['Año'] = datos_noches_reservas_año_verano['Año']
ingresos_por_estancia['Ingresos Anuales'] = datos_noches_reservas_año_verano['Media Estancia'] * precio_medio['Precio Medio Anual']
ingresos_por_estancia['Ingresos Verano'] = datos_noches_reservas_año_verano['Media Estancia Verano'] * precio_medio['Precio Medio Verano']
ingresos_por_estancia = ingresos_por_estancia.fillna(0)
ingresos_por_estancia

In [ ]:
años = ingresos_por_estancia['Año']
ingresos_estancia_anual = ingresos_por_estancia['Ingresos Anuales']
ingresos_estancia_verano = ingresos_por_estancia['Ingresos Verano']

bar_width = 0.35
r1 = np.arange(len(años))
r2 = [x + bar_width for x in r1]
plt.figure(figsize=(16, 7))

plt.bar(r1, ingresos_estancia_anual, color='b', width=bar_width, edgecolor='grey', label='Ingresos por estancia anual')
plt.bar(r2, ingresos_estancia_verano, color='#EB984E', width=bar_width, edgecolor='grey', label='Ingresos por estancia junio a agosto')

plt.xlabel('Año', fontweight='bold', fontsize=12)
plt.ylabel('Ingresos Medios', fontsize=12)
plt.title('Ingresos medios por estancia', fontsize=16)
plt.xticks([r + bar_width / 2 for r in range(len(años))], años, fontsize=12)  # Ajustar etiquetas para que se centren entre las barras

fig = plt.gcf()
fig.subplots_adjust(right=0.75)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)
plt.tight_layout()

plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/ingresos_medios_estancia.pdf', dpi=1000, format='pdf')
plt.show()

GRAFICA DE EVOLUCION DE OCUPACION EN UN AÑO PERO CON TODOS LOS DATOS DEL PERIODO

In [ ]:
datos_meses2_reservados = datos_meses2[datos_meses2['Occupancy Rate'] > 0]
datos_meses2_reservados['Reporting Month'] = pd.to_datetime(datos_meses2_reservados['Reporting Month'])
datos_meses2_reservados['Mes'] = datos_meses2_reservados['Reporting Month'].dt.month
datos_meses2_reservados

In [ ]:
datos_meses2_reservados_por_mes = datos_meses2_reservados[['Occupancy Rate', 'Mes']].groupby('Mes').mean()
datos_meses2_reservados_por_mes

In [ ]:
datos_meses2_reservados_por_mes['Occupancy Rate'].mean()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

datos_meses2_reservados_por_mes = datos_meses2_reservados_por_mes.sort_index()
datos_meses2_reservados_por_mes['Occupancy Rate'] *= 100

x = np.arange(len(datos_meses2_reservados_por_mes))
y = datos_meses2_reservados_por_mes['Occupancy Rate'].values

#  máximo y mínimo
max_index = y.argmax()
min_index = y.argmin()

#curva
x_new = np.linspace(x.min(), x.max(), 300)

spl = make_interp_spline(x, y, k=3) #revisar
y_smooth = spl(x_new)

plt.figure(figsize=(12, 6))
plt.plot(x_new, y_smooth, label='Occupancy Rate', color='#3498db', linewidth=2, zorder=1)

plt.scatter(x[max_index], y[max_index], color='#2ecc71', s=100, label=f'Máximo: {y[max_index]:.2f}%', zorder=2)
plt.scatter(x[min_index], y[min_index], color='#e74c3c', s=100, label=f'Mínimo: {y[min_index]:.2f}%', zorder=2)

plt.scatter(x, y, color='#34495e', marker='o', zorder=1)

plt.xlabel('Mes', fontsize=14, fontweight='bold', color='#34495e')
plt.ylabel('Occupancy Rate (%)', fontsize=14, fontweight='bold', color='#34495e')
plt.title('Average Occupancy Rate by Month', fontsize=18, fontweight='bold', color='#34495e')
plt.xticks(x, datos_meses2_reservados_por_mes.index, rotation=45, fontsize=10, color='#34495e')

plt.ylim(0, 100)
plt.grid(True, linestyle='--', alpha=0.7)

plt.gca().set_facecolor('#f5f5f5')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('#95a5a6')
plt.gca().spines['bottom'].set_color('#95a5a6')

plt.legend(fontsize=12, loc='upper right')

plt.annotate(f'agosto: {y[max_index]:.2f}%', (x[max_index], y[max_index]), xytext=(x[max_index] + 0.5, y[max_index] + 5),
             fontsize=12, color='#2ecc71', fontweight='bold', arrowprops=dict(arrowstyle='->', color='#2ecc71'))
plt.annotate(f'noviembre: {y[min_index]:.2f}%', (x[min_index], y[min_index]), xytext=(x[min_index] + 0.5, y[min_index] - 8),
             fontsize=12, color='#e74c3c', fontweight='bold', arrowprops=dict(arrowstyle='->', color='#e74c3c'))

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/occupancy_rate_by_month_smooth_with_max_min_beautiful.pdf', dpi=300, format='pdf')
plt.show()




IGUAL PERO SOLO DE LOS ÚLTIMOS 12 MESES

In [ ]:
fecha_mas_reciente = datos_meses2_reservados['Reporting Month'].max()
fecha_limite = fecha_mas_reciente - pd.DateOffset(months=12)
ultimos_12_meses = datos_meses2_reservados[datos_meses2_reservados['Reporting Month'] >= fecha_limite]
ultimos_12_meses

In [ ]:
datos_meses2_reservados_por_mes_ultimos_12 = ultimos_12_meses[['Occupancy Rate', 'Mes']].groupby('Mes').mean()
datos_meses2_reservados_por_mes_ultimos_12

# MAPA DE ALOJAMIENTOS

In [ ]:
pip install pdfkit

In [ ]:
m = folium.Map(location=[datos_properties_columnas_completadas_merge['Latitude'].mean(),
                         datos_properties_columnas_completadas_merge['Longitude'].mean()],
               zoom_start=6)
marker_cluster = MarkerCluster().add_to(m)

for index, row in datos_properties_columnas_completadas_merge.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row.get('Nombre', 'Sin nombre')
    ).add_to(marker_cluster)

display(m)

# EVENTOS TORREVIEJA


DATOS EVENTOS

In [ ]:
eventos = pd.read_csv("/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/EVENTOS.csv")
eventos

In [ ]:
eventos['Total Recaudado'] = None
eventos['Tasa Ocupacion'] = None
eventos['Total Diario'] = None

AGRUPADO POR DÍA, EL PRECIO TOTAL INGRESADO Y EL NÚMERO DE ALOJAMIENTOS RESERVADOS ESE DIA, CREOOOOOO

In [ ]:
datos_diarios_reservados_properties = pd.merge(datos_diarios_reservados, datosAlojamientos,on='Property ID', how='inner')

datos_diarios_reservados_fecha_aux = datos_diarios_reservados_properties
datos_diarios_reservados_fecha_aux['Conteo'] = 1
datos_diarios_reservados_fecha_aux = datos_diarios_reservados_fecha_aux.groupby('Date')
datos_diarios_reservados_fecha_aux.sum().reset_index()[['Date', 'Price (Native)', 'Conteo']]

In [ ]:
datos_diarios_reservados_fecha_aux['Date'].dtype
datos_diarios_reservados_fecha_aux_aux = datos_diarios_reservados_fecha_aux.sum().reset_index()[['Date', 'Price (Native)', 'Conteo']]
datos_diarios_reservados_fecha_aux_aux['Date'] = pd.to_datetime(datos_diarios_reservados_fecha_aux_aux['Date'])
datos_diarios_reservados_fecha_aux_aux[datos_diarios_reservados_fecha_aux_aux['Date'] > '2023-08']

In [ ]:
datos_diarios23 = pd.merge(datos_diarios, datosAlojamientos, on='Property ID', how='inner')
datos_diarios233 = datos_diarios23
ocupacion_diaria23 = datos_diarios233.groupby(['Date', 'Status']).size().unstack(fill_value=0)
ocupacion_diaria23['tasa_ocupacion'] = ocupacion_diaria23['R'] / (ocupacion_diaria23['A'] + ocupacion_diaria23['R'])
ocupacion_diaria23.reset_index(inplace=True)
ocupacion_diaria23

In [ ]:
ocupacion_diaria23['Date'] = pd.to_datetime(ocupacion_diaria23['Date'] )

In [ ]:
fecha = '2016-09-04'
fe = pd.to_datetime(fecha)
print(type(fe.strftime('%Y-%m-%d')))
ocupacion_diaria23[ocupacion_diaria23['Date'] == fe.strftime('%Y-%m-%d')]['tasa_ocupacion'].iloc[0]

In [ ]:
from datetime import datetime, timedelta

totales_diario = datos_diarios_reservados_fecha_aux.sum().reset_index()

totales_diario['Date'] = pd.to_datetime(totales_diario['Date'])



for evento, fila in eventos.iterrows():
    evento = fila['NOMBRE EVENTO Y EDICION']
    fi = datetime.strptime(fila['FECHA INICIO'], "%Y-%m-%d").date()-timedelta(days=1)
    ff = datetime.strptime(fila['FECHA FIN'], "%Y-%m-%d").date()



    fecha_inicio = pd.to_datetime(fi)
    fecha_fin = pd.to_datetime(ff)

    print("inicio: " + str(fecha_inicio) + " fin: " + str(fecha_fin))

    #HASTA AQUI TENGO PARA CADA EVENTO LA FECHA INICIO Y FECHA FIN
    #AHORA TENGO QUE VER LA DIFERENCIA DE DÍAS ENTRE ESTAS FECHA
    #PARA ASI SABER QUE FECHAS TENGO QUE BUSCAR EN EL DATASET
    #Y UNA VEZ ENCUENTRE LA FECHA FIN +1 O +2 PARAR EL BUCLE PORQUE ES INNECESARIO

    #EMPEZAR 1 DÍA ANTES DEL DÍA INICIO Y AÑADIR 1 DÍA MAS AL FIN

    #HACER UN ARRAY DE FECHAS Y CON ISIN ENCONTRAR ESAS FECHAS Y ASÍ PODER SUMAR EL TOTAL

    fechas = []
    fechas.append(fecha_inicio.strftime('%Y-%m-%d'))

    total_evento = 0
    total_ocupacion = 0
    ocupacion_diaria = 0

    while fecha_inicio <= fecha_fin:


      #AHORA YA SOLO NOS QUEDA SACAR EL SUMATORIO DE CADA EVENTO, ESTARIA BIEN HACER UN DATAFRAME CON ESTOS DATOS
      #Y ASÍ SE QUEDA MEJOR ESTRUCTURADO
      print("hola")
      print(totales_diario['Date'].dtypes)


      #aqui me guarda el total diario para cada fecha que comparo
      total_diario = totales_diario[totales_diario['Date'] == fecha_inicio]['Price (Native)'].values

      print(total_diario)

      #llamada a la funcion para calcular la ocupacion
      print("Probando " + str(fecha_inicio))
      print(type(fecha_inicio))
      fecha2 = fecha_inicio.strftime('%Y-%m-%d')
      print("Probando " + str(fecha2))
      ocupacionnn = ocupacion_diaria23[ocupacion_diaria23['Date'] == fecha2]['tasa_ocupacion'].iloc[0]
      ocupacion_diaria = ocupacion_diaria + ocupacionnn
      #funcion_ocupacion(fecha_inicio)


      fecha_inicio += timedelta(days=1)
      fechas.append(fecha_inicio.strftime('%Y-%m-%d'))

      #DARLE UNA VUELTAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
      #filas_dias = datos_diarios_reservados.sum().reset_index()[datos_diarios_reservados.sum().reset_index()['Date'].isin(fechas)]
      #filas_dias

      #SOBRE ESTE DATAFRAME CREO QUE TENGO QUE ITERAR PARA SACAR EL SUMATORIO


      dias_evento = fecha_fin - fecha_inicio

      total_evento = total_evento + total_diario

    print("Total evento: "+ str(total_evento))
    #añadir al evento el total generado

    total_ocupacion = ocupacion_diaria/len(fechas)

    total_diario = total_evento/len(fechas)

    eventos.loc[eventos['NOMBRE EVENTO Y EDICION'] == evento, 'Total Recaudado'] = total_evento
    eventos.loc[eventos['NOMBRE EVENTO Y EDICION'] == evento, 'Tasa Ocupacion'] = total_ocupacion
    eventos.loc[eventos['NOMBRE EVENTO Y EDICION'] == evento, 'Total Diario'] = total_diario

      #print(fechas)

    #COGER 1 DÍA ANTEs Y UN DÍA DESPUES






In [ ]:
#datos_diarios2 = pd.merge(datos_diarios, datosAlojamientos, on='Property ID', how='inner')
datos_dairios_aux = datos_diarios23
datos_dairios_aux['Date'] = pd.to_datetime(datos_dairios_aux['Date'])

#MEJORA: CREO 2 DATASETS, UNO CON A Y OTRO CON R, AGRUPO POR DIA Y VER COMO SACAR LOS APARTAMENTOS PUESTO EN ALQUILER

def funcion_ocupacion(fecha):

  #cogemos valores del datos_diarios
  #deolvemos ya la tasa de ocupación de ese día
  #así en la otra función se van sumando y se divide ese total entre los días totales del array

  valores_ocupacion = ['A', 'R']
  valores_reservados = ['R']
  #ocupacion = datos_dairios_aux[datos_dairios_aux['Status'].isin(valores)]
  #ocupacion

  ofertados = datos_dairios_aux[(datos_dairios_aux['Status'].isin(valores_ocupacion)) & (datos_dairios_aux['Date'] == fecha)]
  apt_ofertados = ofertados.shape[0]

  reservados = datos_dairios_aux[(datos_dairios_aux['Status'] == 'R') & (datos_dairios_aux['Date'] == fecha)]
  apt_reservados = reservados.shape[0]

  return apt_reservados/apt_ofertados

COMPARACIÓN TASAS DE OCUPACIÓN CUANDO SON

In [ ]:
import ast

eventos_comparar = pd.read_csv("/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/EVENTOS-COMPARAR.csv")
eventos_comparar['FECHA NO ORGANIZADO'] = eventos_comparar['FECHA NO ORGANIZADO'].apply(ast.literal_eval)
eventos_comparar

In [ ]:
eventos_comparar['TASA DE OCUPACION'] = 0
eventos_comparar['TASA DE OCUPACION MEDIA'] = 0

In [ ]:
from datetime import datetime, timedelta

for evento_comparar, fila in eventos_comparar.iterrows():
    media = 0
    evento_comparar = fila['NOMBRE EVENTO Y EDICION']
    print(evento_comparar)
    fi = datetime.strptime(fila['FECHA INICIO'], "%Y-%m-%d").date()-timedelta(days=1)
    ff = datetime.strptime(fila['FECHA FIN'], "%Y-%m-%d").date()

    fechas_no_organizado = fila['FECHA NO ORGANIZADO']

    total_ocupacion_año = 0
    total_ocupacion_evento = 0

    for año in fechas_no_organizado:
        #en año tengo los años en que no se han organizado
        print(año)

        fecha_inicio = pd.to_datetime(fi)
        fecha_inicio_dia = fecha_inicio.day
        fecha_inicio_mes = fecha_inicio.month
        fecha_inicio_comprar = datetime(year=año, month=fecha_inicio_mes, day=fecha_inicio_dia)

        print(fecha_inicio_comprar)


        fecha_fin = pd.to_datetime(ff)
        fecha_fin_dia = fecha_fin.day
        fecha_fin_mes = fecha_fin.month
        fecha_fin_comprar = datetime(year=año, month=fecha_fin_mes, day=fecha_fin_dia)

        print(fecha_fin_comprar)

        fechas = []
        fechas.append(fecha_inicio_comprar.strftime('%Y-%m-%d'))
        ocupacion_diaria = 0

        while fecha_inicio_comprar <= fecha_fin_comprar:

            fecha2 = fecha_inicio_comprar.strftime('%Y-%m-%d')
            print("Probando " + str(fecha2))
            ocupacionnn = ocupacion_diaria23[ocupacion_diaria23['Date'] == fecha2]['tasa_ocupacion'].iloc[0]

            ocupacion_diaria = ocupacion_diaria + ocupacionnn
            #funcion_ocupacion(fecha_inicio_comprar)

            fechas.append(fecha_inicio_comprar.strftime('%Y-%m-%d'))
            fecha_inicio_comprar += timedelta(days=1)

        total_ocupacion_año = ocupacion_diaria/len(fechas)
        total_ocupacion_evento = total_ocupacion_evento + total_ocupacion_año
        print(total_ocupacion_año)

    media = total_ocupacion_evento/len(fechas_no_organizado)
    eventos_comparar.loc[eventos_comparar['NOMBRE EVENTO Y EDICION'] == evento_comparar,'TASA DE OCUPACION MEDIA'] = media
    filtro = eventos[eventos['NOMBRE EVENTO Y EDICION'] == evento_comparar]
    print(eventos['NOMBRE EVENTO Y EDICION'])
    print(filtro['Tasa Ocupacion'])
    eventos_comparar.loc[eventos_comparar['NOMBRE EVENTO Y EDICION'] == evento_comparar,'TASA DE OCUPACION'] = filtro['Tasa Ocupacion'].iloc[0]
    print("Media ocupacion" + str(media))



In [ ]:
eventos_comparar

In [ ]:
eventos_comparar['DIFERENCIA TASA DE OCUPACION'] = eventos_comparar['TASA DE OCUPACION'] - eventos_comparar['TASA DE OCUPACION MEDIA']
eventos_comparar['DIFERENCIA TASA DE OCUPACION'] = eventos_comparar['DIFERENCIA TASA DE OCUPACION'].round(2)
eventos_comparar['TASA DE OCUPACION'] = eventos_comparar['TASA DE OCUPACION'].round(2)
eventos_comparar['TASA DE OCUPACION MEDIA'] = eventos_comparar['TASA DE OCUPACION MEDIA'].round(2)
eventos_comparar['PORCENTAJE AUMENTO'] = (eventos_comparar['TASA DE OCUPACION']-eventos_comparar['TASA DE OCUPACION MEDIA'])/eventos_comparar['TASA DE OCUPACION MEDIA']*100
eventos_comparar['PORCENTAJE AUMENTO'] = eventos_comparar['PORCENTAJE AUMENTO'].round(2)
eventos_comparar

In [ ]:
eventos_ordenado_diferencia = eventos_comparar.sort_values(by='DIFERENCIA TASA DE OCUPACION', ascending=False)
eventos_ordenado_diferencia.drop(6, inplace=True)
plt.figure(figsize=(22, 10))
sns.barplot(y='NOMBRE EVENTO Y EDICION', x='DIFERENCIA TASA DE OCUPACION', data=eventos_ordenado_diferencia, width=0.5)
plt.title('Ranking eventos ordenados por diferencia de tasa de ocupación', fontsize = 20, pad = 10)
plt.ylabel('Eventos', fontsize=12)
plt.xlabel('Aumento tasa de ocupación')

def to_percent(x, position):
    return f'{x * 100:.0f}%'


plt.yticks(fontsize=16)
plt.xticks(fontsize=10)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/ranking_diferencia_tasa.pdf', dpi=3000, format='pdf')

In [ ]:
eventos_ordenado_diferencia = eventos_comparar.sort_values(by='PORCENTAJE AUMENTO', ascending=False)
eventos_ordenado_diferencia.drop(6, inplace=True)
plt.figure(figsize=(22, 10))
sns.barplot(y='NOMBRE EVENTO Y EDICION', x='PORCENTAJE AUMENTO', data=eventos_ordenado_diferencia, width=0.5)
plt.title('Ranking eventos ordenados por % de aumento de ocupación', fontsize = 20, pad = 10)
plt.ylabel('Eventos')
plt.xlabel('Porcentaje de aumento de ocupación')
plt.yticks(fontsize=16)
plt.xticks(fontsize=10)
ticks_x = plt.gca().get_xticks()

labels_x = ['{}%'.format(int(x)) for x in ticks_x]

plt.gca().set_xticklabels(labels_x)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/ranking_aumento_ocupacion.pdf', dpi=3000, format='pdf')

# RANKINGS

In [ ]:
eventos

In [ ]:
eventos.to_csv('/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/eventos_final.pdf', index=False)

In [ ]:
import matplotlib.ticker as mtick
eventos_ordenado = eventos.sort_values(by='Total Recaudado', ascending=False)
plt.figure(figsize=(24, 24))
sns.barplot(y='NOMBRE EVENTO Y EDICION', x='Total Recaudado',data=eventos_ordenado)
plt.title('Ranking eventos ordenados por total ingresado', fontsize = 20, pad=10)
plt.xlabel('Total recaudado (en millones)')
plt.ylabel('Nombre del evento y edición')
plt.yticks(fontsize=16)
plt.xticks(fontsize=10)
plt.gca().xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{int(x / 1e6)}M'))
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/ranking_eventos_ingresos_totales.pdf', dpi=3000, format='pdf')

In [ ]:
eventos_ordenado = eventos.sort_values(by='Tasa Ocupacion', ascending=False)
plt.figure(figsize=(24, 24))
sns.barplot(y='NOMBRE EVENTO Y EDICION', x='Tasa Ocupacion',data=eventos_ordenado)
plt.title('Ranking Eventos ordenados por tasa de ocupación', fontsize = 20, pad=10)
plt.xlabel('Tasa de ocupación media')
plt.ylabel('Nombre del evento y edición')
plt.yticks(fontsize=16)
plt.xticks(fontsize=10)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/ranking_eventos_tasa_ocupacion.pdf', dpi=3000, format='pdf')

In [ ]:
eventos_ordenado = eventos.sort_values(by='Total Diario', ascending=False)
plt.figure(figsize=(24, 24))
sns.barplot(y='NOMBRE EVENTO Y EDICION', x='Total Diario',data=eventos_ordenado)
plt.title('Ranking Eventos ordenados por ingresos diarios', fontsize = 20, pad=10)
plt.xlabel('Total recaudado diario')
plt.ylabel('Nombre del evento y edición')
plt.yticks(fontsize=16)
plt.xticks(fontsize=10)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/ranking_eventos_ingresos_diarios.pdf', dpi=3000, format='pdf')

DATOS DIARIOS PARA CONOCER OCUPACIÓN...

In [ ]:
#datos diario
#properties_final
valores = ['A', 'R']
ocupacion = datos_diarios[datos_diarios['Status'].isin(valores)]
ocupacion

fecha_concreta = '2021-07-31'

ofertados = datos_diarios[(datos_diarios['Status'].isin(valores)) & (datos_diarios['Date'] == fecha_concreta)]
apt_ofertados = ofertados.shape[0]

reservados = datos_diarios[(datos_diarios['Status'] == 'R') & (datos_diarios['Date'] == fecha_concreta)]
apt_reservados = reservados.shape[0]

apt_reservados/apt_ofertados

#combinacion = pd.merge(datos_diarios, properties_final, how='inner')
#combinacion.groupby('Property ID').sum()

TOTAL RECAUDADO POR AÑO

In [ ]:
eventos_año_total_recaudado = eventos.groupby('AÑO')['Total Recaudado'].sum().reset_index()
eventos_año_total_recaudado

# NUEVAS GRÁFICAS DE EVENTOS

In [ ]:
eventos_final = pd.read_csv('/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/eventos_final.pdf')
eventos_final_grafica = eventos_final[['AÑO', 'NOMBRE EVENTO', 'Total Diario']]
eventos_final_grafica['Total Diario'] = eventos_final_grafica['Total Diario'].astype(int)
eventos_final_grafica

In [ ]:
eventos_final

In [ ]:
eventos_final_grafica[['Total Diario', 'AÑO']].groupby('AÑO').sum()

In [ ]:
eventos_final_grafica[['NOMBRE EVENTO', 'Total Diario']].groupby('NOMBRE EVENTO').sum()


In [ ]:
eventos_final_grafica.loc[eventos_final_grafica['NOMBRE EVENTO'] == 'CERTAMEN INTERNACIONAL DE HABANERAS Y POLIFONÍA', 'NOMBRE EVENTO'] = 'CERT. INT. HABANERAS Y POLIFONÍA'
eventos_final_grafica.loc[eventos_final_grafica['NOMBRE EVENTO'] == 'CERTAMEN JUVENIL DE HABANERAS Y POLIFONÍA', 'NOMBRE EVENTO'] = 'CERT. JUEVNIL HABANERAS Y POLIFONÍA'
eventos_final_grafica.loc[eventos_final_grafica['NOMBRE EVENTO'] == 'EUROMARINA OPTIMISTIC DE VEL', 'NOMBRE EVENTO'] = 'EUROMARINA OPTIMISTIC'

eventos_final.loc[eventos_final['NOMBRE EVENTO'] == 'CERTAMEN INTERNACIONAL DE HABANERAS Y POLIFONÍA', 'NOMBRE EVENTO'] = 'CERT. INT. HABANERAS Y POLIFONÍA'
eventos_final.loc[eventos_final['NOMBRE EVENTO'] == 'CERTAMEN JUVENIL DE HABANERAS Y POLIFONÍA', 'NOMBRE EVENTO'] = 'CERT. JUEVNIL HABANERAS Y POLIFONÍA'
eventos_final.loc[eventos_final['NOMBRE EVENTO'] == 'EUROMARINA OPTIMISTIC DE VEL', 'NOMBRE EVENTO'] = 'EUROMARINA OPTIMISTIC'

eventos_final_grafica

In [ ]:
plt.figure(figsize=(12, 8))

eventos_unicos = eventos_final_grafica['NOMBRE EVENTO'].unique()

for evento in eventos_unicos:
    df_evento = eventos_final_grafica[eventos_final_grafica['NOMBRE EVENTO'] == evento]
    plt.plot(df_evento['AÑO'], df_evento['Total Diario'], marker='o', label=evento)

plt.xlabel('Año')
plt.ylabel('Total Diario')
plt.title('Total Diario por Año y Evento')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
eventos_unicos = eventos_final_grafica['NOMBRE EVENTO'].unique()
años_unicos = sorted(eventos_final_grafica['AÑO'].unique())

totals = {evento: [0] * len(años_unicos) for evento in eventos_unicos}

for index, row in eventos_final_grafica.iterrows():
    reseteo = True
    evento = row['NOMBRE EVENTO']
    año = row['AÑO']
    total_diario = row['Total Diario']
    idx_año = años_unicos.index(año)

    if index > 0:
        fila_anterior = eventos_final_grafica.iloc[index - 1]
        if evento != fila_anterior['NOMBRE EVENTO']:
          reseteo = True

    if index == 0 or reseteo == True:
        totals[evento][idx_año] = total_diario
    else:
        totals[evento][idx_año] = totals[evento][idx_año - 1] + total_diario

plt.figure(figsize=(12, 8))

bottom = None
for evento in eventos_unicos:
    plt.bar(años_unicos, totals[evento], label=evento, bottom=bottom)
    if bottom is None:
        bottom = [0] * len(años_unicos)
    bottom = [bottom[i] + totals[evento][i] for i in range(len(años_unicos))]

plt.xlabel('Evento')
plt.ylabel('Total Diario Acumulado')
plt.title('Total Diario Acumulado por Evento y Año')
plt.xticks(años_unicos)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

eventos_final_grafica['Total Diario'] = eventos_final_grafica['Total Diario'].astype(int)
eventos_unicos = eventos_final_grafica['NOMBRE EVENTO'].unique()
años_unicos = sorted(eventos_final_grafica['AÑO'].unique())

max_totales = eventos_final_grafica.groupby('NOMBRE EVENTO')['Total Diario'].max()

eventos_ordenados_ingresos = max_totales.sort_values(ascending=True).index.tolist()

totals = {año: [0] * len(eventos_ordenados_ingresos) for año in años_unicos}

for index, row in eventos_final_grafica.iterrows():
    reseteo = True
    evento = row['NOMBRE EVENTO']
    año = row['AÑO']
    total_diario = row['Total Diario']
    idx_evento = eventos_ordenados_ingresos.index(evento)

    if index > 0:
        fila_anterior = eventos_final_grafica.iloc[index - 1]
        if evento != fila_anterior['NOMBRE EVENTO']:
            reseteo = True

    if index == 0 or reseteo == True:
        totals[año][idx_evento] = total_diario
    else:
        totals[año][idx_evento] = totals[año][idx_evento - 1] + total_diario


plt.figure(figsize=(12, 8))
bottom = [0] * len(eventos_ordenados_ingresos)
for año in años_unicos:
    plt.barh(eventos_ordenados_ingresos, totals[año], label=año, left=bottom)
    bottom = [bottom[i] + totals[año][i] for i in range(len(eventos_ordenados_ingresos))]


plt.ylabel('Evento')
plt.xlabel('Total Diario Acumulado')
plt.title('Total Diario Acumulado por Evento y Año')
plt.yticks(rotation=0)
plt.legend(title='Año', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/rankin_eventos_stack_bar.pdf', dpi=3000, format='pdf')

ORDENARLO: COGER EL MÁXIMO DIARIO DE CADA EVENTO Y SE ORDENAN DESCENDENTEMENTE

IDEA: DESGLOSAR BRILLA TORREVIEJA PARA VER QUÉ ARTISTA RECAUDA MÁS O AUMENTA MÁS LA OCUPACIÓN RESPECTO AL RESTO

In [ ]:
import matplotlib.pyplot as plt

eventos_final['Tasa Ocupacion'] = eventos_final['Tasa Ocupacion'].astype(float)
eventos_unicos = eventos_final['NOMBRE EVENTO'].unique()
años_unicos = sorted(eventos_final['AÑO'].unique())

max_totales = eventos_final.groupby('NOMBRE EVENTO')['Tasa Ocupacion'].max()

#eventos_ordenados = max_totales.sort_values(ascending=True).index.tolist()

totals = {año: [0] * len(eventos_ordenados_ingresos) for año in años_unicos}

for index, row in eventos_final.iterrows():
    reseteo = True
    evento = row['NOMBRE EVENTO']
    año = row['AÑO']
    total_ocupacion = row['Tasa Ocupacion']
    idx_evento = eventos_ordenados_ingresos.index(evento)

    if index > 0:
        fila_anterior = eventos_final.iloc[index - 1]
        if evento != fila_anterior['NOMBRE EVENTO']:
            reseteo = True

    if index == 0 or reseteo == True:
        totals[año][idx_evento] = total_ocupacion
    else:
        totals[año][idx_evento] = totals[año][idx_evento - 1] + total_ocupacion

plt.figure(figsize=(12, 8))

bottom = [0] * len(eventos_ordenados_ingresos)
for año in años_unicos:
    plt.barh(eventos_ordenados_ingresos, totals[año], label=año, left=bottom)
    bottom = [bottom[i] + totals[año][i] for i in range(len(eventos_ordenados_ingresos))]

plt.ylabel('Evento')
plt.xlabel('Total Ocupación Acumulado')
plt.title('Total Ocupación Acumulado por Evento y Año')
plt.yticks(rotation=0)
plt.legend(title='Año', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

plt.savefig('/content/drive/MyDrive/Sergio Naranjo Fernández/FICHEROS/rankin_eventos_stack_bar_ocupacion.pdf', dpi=3000, format='pdf')



ORDENAR POR DIFERENCIAS DE TASAS DE UN AÑO HA OTRO, Y LA QUE DE MEDIA HAYA AUMENTADO MAS ES LA PRIMERA Y ASI SUCESIVAMENTE

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

eventos_final_grafica['Total Diario'] = eventos_final_grafica['Total Diario'].astype(int)

eventos_unicos = eventos_final_grafica['NOMBRE EVENTO'].unique()
años_unicos = sorted(eventos_final_grafica['AÑO'].unique())

totals = {año: [0] * len(eventos_unicos) for año in años_unicos}
total_eventos = [0] * len(eventos_unicos)

for index, row in eventos_final_grafica.iterrows():
    evento = row['NOMBRE EVENTO']
    año = row['AÑO']
    total_diario = row['Total Diario']
    idx_evento = list(eventos_unicos).index(evento)
    totals[año][idx_evento] += total_diario
    total_eventos[idx_evento] += total_diario

percentages = {año: [0] * len(eventos_unicos) for año in años_unicos}
for año in años_unicos:
    for idx_evento in range(len(eventos_unicos)):
        if total_eventos[idx_evento] > 0:
            percentages[año][idx_evento] = (totals[año][idx_evento] / total_eventos[idx_evento]) * 100

plt.figure(figsize=(12, 8))

bottom = [0] * len(eventos_unicos)
for año in años_unicos:
    plt.bar(eventos_unicos, percentages[año], label=año, bottom=bottom)
    for i, percentage in enumerate(percentages[año]):
        if percentage > 0:
            plt.annotate(f'{percentage:.1f}%', xy=(eventos_unicos[i], bottom[i] + percentage / 2), ha='center', va='center', fontsize=8)
    bottom = [bottom[i] + percentages[año][i] for i in range(len(eventos_unicos))]

plt.xlabel('Evento')
plt.ylabel('Contribución Porcentual al Total Diario')
plt.title('Contribución Porcentual al Total Diario por Evento y Año')
plt.xticks(rotation=90)
plt.legend(title='Año', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

plt.show()


# PREPARACION DATOS MODELO PREDICTIVO

In [ ]:
datosPropiedadesPredictivo = pd.merge(datosProperties, datosAlojamientos, on="Property ID", how="inner")
datosPropiedadesPredictivo
#vamos a quitar Created Date, Occupancy Rate LTM, Response Rate, Latitude, Longitude, Año Creacion
columnas_a_eliminar = ['Created Date', 'Occupancy Rate LTM', 'Response Rate', 'Latitude', 'Longitude', 'Año Creacion']

# Eliminar las columnas
datosPropiedadesPredictivo = datosPropiedadesPredictivo.drop(columns=columnas_a_eliminar)
datosPropiedadesPredictivo

AMOS A CALCULAR: PRECIO MEDIO, PRECIO MEDIO VERANO, OCUPACIÓN MEDIA, OCUPACIÓN MEDIA VERANO, NÚMERO MEDIO DE RESERVAS, ESTANCIA MEDIA DE RESERVAS

In [ ]:
datosPrecioMedioDaily = datosDaily.groupby("Property ID")['Price (Native)'].mean()
datosPropiedadesPredictivo = pd.merge(datosPropiedadesPredictivo, datosPrecioMedioDaily, on="Property ID", how="inner")
datosPropiedadesPredictivo = pd.merge(datosPropiedadesPredictivo, estancia_minima, on="Property ID", how="inner")
datosPropiedadesPredictivo

In [ ]:
datosDailyAux = pd.merge(datos_diarios, datosAlojamientos, on="Property ID", how="inner")

In [ ]:

datosDailyAux['Dias Ocupados'] = datosDailyAux['Status'].eq('R').groupby(datosDailyAux['Property ID']).transform('sum')
datosDailyAux['Dias Disponibles'] = datosDailyAux['Status'].eq('A').groupby(datosDailyAux['Property ID']).transform('sum')

# Calcular la tasa de ocupación media para cada alojamiento
datosDailyAux['Tasa de Ocupación Media'] = datosDailyAux['Dias Ocupados'] / (datosDailyAux['Dias Ocupados'] + datosDailyAux['Dias Disponibles'])

# Eliminar columnas auxiliares
datosDailyAux.drop(columns=['Dias Ocupados', 'Dias Disponibles'], inplace=True)

In [ ]:
tasa_ocupacion_media_por_propiedad = datosDailyAux.groupby('Property ID')['Tasa de Ocupación Media'].mean().reset_index()

# Mostrar el resultado
tasa_ocupacion_media_por_propiedad

In [ ]:
datosPropiedadesPredictivo = pd.merge(datosPropiedadesPredictivo, tasa_ocupacion_media_por_propiedad, on="Property ID", how="inner")
datosPropiedadesPredictivo

# MODELO PREDICTIVO FINAL FINAL

In [ ]:
import warnings
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import xgboost as xgb
from math import sqrt

In [ ]:
ultimo = pd.read_csv('/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/df_ultimos_12_meses_revenue.csv')
ultimo

In [ ]:
df_encoded = pd.get_dummies(ultimo, columns=['Listing Type'])
df_encoded.columns

In [ ]:

df_encoded_columnas = ultimo
uepa = df_encoded_columnas.drop(columns=['Property ID','Reporting Month', 'Precio/Habitacion','Revenue (Native)', 'Number of Reservations' , 'Revenue/Bed'
                                ,'Listing Type_2', 'Property Type_2'])
uepa.rename(columns={'Extra People Fee (USD)': 'Extra People'}, inplace=True)
uepa.rename(columns={'Minimum Stay_x': 'Minimun Stay'}, inplace=True)
uepa.info()

In [ ]:
df_encoded_2 = pd.get_dummies(df_encoded, columns=['Property Type'])
df_encoded2.columns

In [ ]:
df_encoded_medio = df_encoded_2

In [ ]:
df_encoded_final = df_encoded_medio.drop(columns=['Property ID','Reporting Month', 'Listing Type_2', 'Property Type_2'])
df_encoded_final.rename(columns={'Extra People Fee (USD)': 'Extra People'}, inplace=True)
df_encoded_final.rename(columns={'Minimum Stay_x': 'Minimun Stay'}, inplace=True)
df_encoded_final.columns

In [ ]:
boolean_columns = df_encoded_final.select_dtypes(include=['bool']).columns
df_encoded_final[boolean_columns] = df_encoded_final[boolean_columns].astype(int)
df_encoded_final

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency

random = RandomForestRegressor(n_estimators = 1000,
            criterion    = 'squared_error',
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,
            random_state = 123)

datosModelo = df_encoded_final.drop(columns=['Revenue/Bed', 'Revenue (Native)', 'Occupancy Rate', 'Reservation Days', 'Precio/Habitacion', 'Number of Reservations'])

xx_servicios = datosModelo.drop(['ADR (Native)'], axis=1)
yx_servicios = datosModelo['ADR (Native)']


X_train_servicios, X_test_servicios, y_train_servicios, y_test_servicios = train_test_split(xx_servicios, yx_servicios, test_size=0.2, random_state=123)

train_data = X_train_servicios.join(y_train_servicios)
test_data = X_test_servicios.join(y_test_servicios)
random.fit(X_train_servicios, y_train_servicios)

y_pred = random.predict(X_test_servicios)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_servicios, y_pred, alpha=0.7)
plt.plot([min(y_test_servicios), max(y_test_servicios)], [min(y_test_servicios), max(y_test_servicios)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

In [ ]:
random.score(X_test_servicios, y_test_servicios)

y_pred = random.predict(X_test_servicios)

mse = mean_squared_error(y_test_servicios, y_pred)
mae = mean_absolute_error(y_test_servicios, y_pred)
rmse = sqrt(mean_squared_error(y_test_servicios, y_pred))
r2 = r2_score(y_test_servicios, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

In [ ]:
datosModelo.info()

In [ ]:
importances = random.feature_importances_
feature_names = xx_servicios.columns
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False).round(6)

print(feature_importance_df)

In [ ]:
comparison_df = pd.DataFrame({'Valor Real': y_test_servicios, 'Predicción': y_pred}, index=y_test_servicios.index)
sample_comparison_df = comparison_df.sample(n=10, random_state=123)

print(sample_comparison_df)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

gb_model = GradientBoostingRegressor(loss='squared_error',    # 'ls' se refiere a least squares regression
                                     learning_rate=0.1,
                                     n_estimators=1000,
                                     subsample=1.0,
                                     criterion='friedman_mse',
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     min_weight_fraction_leaf=0.0,
                                     max_depth=3,
                                     random_state=123)

xx_serviciosG = datosModelo.drop(['ADR (Native)'], axis=1)
yx_serviciosG = datosModelo['ADR (Native)']

X_train_serviciosG, X_test_serviciosG, y_train_serviciosG, y_test_serviciosG = train_test_split(xx_serviciosG, yx_serviciosG, test_size=0.20, random_state=123)
gb_model.fit(X_train_serviciosG, y_train_serviciosG)

y_predG = gb_model.predict(X_test_serviciosG)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_serviciosG, y_predG, alpha=0.7)
plt.plot([min(y_test_serviciosG), max(y_test_serviciosG)], [min(y_test_serviciosG), max(y_test_serviciosG)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

mae = mean_absolute_error(y_test_serviciosG, y_predG)
mse = mean_squared_error(y_test_serviciosG, y_predG)
rmse = mean_squared_error(y_test_serviciosG, y_predG, squared=False)
r2 = r2_score(y_test_serviciosG, y_predG)

print(f'Mean Absolute Error (MAE): {mae:.10f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.10f}')
print(f'R^2 Score: {r2:.10f}')
print(f'Mean Squared Error (MSE): {mse:.10f}')

In [ ]:
comparison_dfG = pd.DataFrame({'Valor Real': y_test_serviciosG, 'Predicción': y_predG}, index=y_test_serviciosG.index)

sample_comparison_dfG = comparison_dfG.sample(n=10, random_state=123)

print(sample_comparison_dfG)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

linear = LinearRegression()

xx_serviciosL = datosModelo.drop(['ADR (Native)'], axis=1)
yx_serviciosL = datosModelo['ADR (Native)']


X_train_serviciosL, X_test_serviciosL, y_train_serviciosL, y_test_serviciosL = train_test_split(xx_serviciosL, yx_serviciosL, test_size=0.20, random_state=123)

train_dataL = X_train_serviciosL.join(y_train_serviciosL)
test_dataL = X_test_serviciosL.join(y_test_serviciosL)
linear.fit(X_train_serviciosL, y_train_serviciosL)

y_predL = linear.predict(X_test_serviciosL)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_serviciosL, y_predL, alpha=0.7)
plt.plot([min(y_test_serviciosL), max(y_test_serviciosL)], [min(y_test_serviciosL), max(y_test_serviciosL)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

In [ ]:
linear.score(X_test_serviciosL, y_test_serviciosL)

y_pred = linear.predict(X_test_serviciosL)

mse = mean_squared_error(y_test_serviciosL, y_predL)
mae = mean_absolute_error(y_test_serviciosL, y_predL)
rmse = sqrt(mean_squared_error(y_test_serviciosL, y_predL))
r2 = r2_score(y_test_serviciosL, y_predL)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

In [ ]:
comparison_dfL = pd.DataFrame({'Valor Real': y_test_serviciosL, 'Predicción': y_predL}, index=y_test_serviciosL.index)
sample_comparison_dfL = comparison_dfL.sample(n=10, random_state=123)

print(sample_comparison_dfL)

# 5760 ALOJAMIENTOS ÚLTIMOS 12 MESES

In [ ]:
agrupando = pd.read_csv('/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/df_ultimos_12_meses_revenue.csv')
agrupando

agrupando = agrupando.drop(columns=['Reporting Month', 'Temporada', 'Mes'])
agrupando = agrupando.drop(columns=['Precio/Habitacion', 'Number of Reservations', 'Occupancy Rate', 'Revenue (Native)', 'Reservation Days', 'Revenue/Bed', 'Listing Type_2', 'Property Type_2'])
agrupando.info()

In [ ]:
agg_funs = {
    'ADR (Native)': 'mean',
    'Bedrooms' : 'first',
    'Bathrooms' : 'first',
    'Max Guests' : 'first',
    'Extra People Fee (USD)' : 'first',
    'Number of Photos' : 'first',
    'Minimum Stay_x' : 'first',
    'Pets Allowed' : 'first',
    'Pool' : 'first',
    'Lift/Elevator' : 'first',
    'Parking' : 'first',
    'Internet' : 'first',
    'TV' : 'first',
    'Private Pool': 'first',
    'Air' : 'first',
    'dishwasher' : 'first',
    'doorman_entry' : 'first',
    'ENTERTAINMENT_ALL_AGES|Books': 'first',
    'fireplace' : 'first',
    'Bath/Bed' : 'first',
    'Bed/Bath' : 'first',
    'Listing Type' : 'first',
    'Property Type' : 'first'
}

ultimo_final_agg = agrupando.groupby('Property ID').agg(agg_funs).reset_index()
ultimo_final_agg

SOLO PRECIO MEDIO TEMPORADA BAJA

In [ ]:
agrupando2 = pd.read_csv('/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/df_ultimos_12_meses_revenue.csv')
agrupando_baja = agrupando2[(agrupando2['Mes'] < 6) | (agrupando2['Mes'] > 8 )]
agrupando_baja
agrupando_baja = agrupando_baja.drop(columns=['Reporting Month', 'Temporada', 'Mes','Precio/Habitacion'])
agrupando_baja.info()

agg_funs = {
    'ADR (Native)': 'mean',
    'Bedrooms' : 'first',
    'Bathrooms' : 'first',
    'Max Guests' : 'first',
    'Extra People Fee (USD)' : 'first',
    'Number of Photos' : 'first',
    'Minimum Stay_x' : 'first',
    'Pets Allowed' : 'first',
    'Pool' : 'first',
    'Lift/Elevator' : 'first',
    'Parking' : 'first',
    'Internet' : 'first',
    'TV' : 'first',
    'Private Pool': 'first',
    'Air' : 'first',
    'dishwasher' : 'first',
    'doorman_entry' : 'first',
    'ENTERTAINMENT_ALL_AGES|Books': 'first',
    'fireplace' : 'first',
    'Bath/Bed' : 'first',
    'Bed/Bath' : 'first',
    'Listing Type' : 'first',
    'Property Type' : 'first'
}

ultimo_final_agg_baja = agrupando_baja.groupby('Property ID').agg(agg_funs).reset_index()
ultimo_final_agg_baja

In [ ]:
ultimo_final_agg_baja_one_hot = pd.get_dummies(ultimo_final_agg_baja, columns=['Listing Type'])

In [ ]:
ultimo_final_agg_baja_one_hot2 = pd.get_dummies(ultimo_final_agg_baja_one_hot, columns=['Property Type'])

In [ ]:
ultimo_final_agg_baja_one_hot2.info()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

randomAgrupado = RandomForestRegressor(n_estimators = 1000,
            criterion    = 'squared_error',
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,
            random_state = 123)

datosModeloAgrupado = ultimo_final_agg_baja_one_hot2
datosModeloAgrupado = datosModeloAgrupado.drop(columns=['Property ID'])


xx_serviciosRA = datosModeloAgrupado.drop(['ADR (Native)'], axis=1)
yx_serviciosRA = datosModeloAgrupado['ADR (Native)']


X_train_serviciosRA, X_test_serviciosRA, y_train_serviciosRA, y_test_serviciosRA = train_test_split(xx_serviciosRA, yx_serviciosRA, test_size=0.20, random_state=123)
randomAgrupado.fit(X_train_serviciosRA, y_train_serviciosRA)

y_predRA = randomAgrupado.predict(X_test_serviciosRA)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_serviciosRA, y_predRA, alpha=0.7)
plt.plot([min(y_test_serviciosRA), max(y_test_serviciosRA)], [min(y_test_serviciosRA), max(y_test_serviciosRA)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

In [ ]:
randomAgrupado.score(X_test_serviciosRA, y_test_serviciosRA)

y_predRA = randomAgrupado.predict(X_test_serviciosRA)

mse = mean_squared_error(y_test_serviciosRA, y_predRA)
mae = mean_absolute_error(y_test_serviciosRA, y_predRA)
rmse = sqrt(mean_squared_error(y_test_serviciosRA, y_predRA))
r2 = r2_score(y_test_serviciosRA, y_predRA)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
gb_modelAgrupado = GradientBoostingRegressor(loss='squared_error',
                                     learning_rate=0.1,
                                     n_estimators=1000,
                                     subsample=1.0,
                                     criterion='friedman_mse',
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     min_weight_fraction_leaf=0.0,
                                     max_depth=3,
                                     random_state=123)

xx_serviciosGA = datosModeloAgrupado.drop(['ADR (Native)'], axis=1)
yx_serviciosGA = datosModeloAgrupado['ADR (Native)']

X_train_serviciosGA, X_test_serviciosGA, y_train_serviciosGA, y_test_serviciosGA = train_test_split(xx_serviciosGA, yx_serviciosGA, test_size=0.20, random_state=123)

scaler = StandardScaler()
X_train_servicios_scaledGA = scaler.fit_transform(X_train_serviciosGA)
X_test_servicios_scaledGA = scaler.transform(X_test_serviciosGA)

gb_modelAgrupado.fit(X_train_servicios_scaledGA, y_train_serviciosGA)

y_predGA = gb_modelAgrupado.predict(X_test_servicios_scaledGA)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_serviciosGA, y_predGA, alpha=0.7)
plt.plot([min(y_test_serviciosGA), max(y_test_serviciosGA)], [min(y_test_serviciosGA), max(y_test_serviciosGA)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

mae = mean_absolute_error(y_test_serviciosGA, y_predGA)
mse = mean_squared_error(y_test_serviciosGA, y_predGA)
rmse = mean_squared_error(y_test_serviciosGA, y_predGA, squared=False)
r2 = r2_score(y_test_serviciosGA, y_predGA)

print(f'Mean Absolute Error (MAE): {mae:.10f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.10f}')
print(f'R^2 Score: {r2:.10f}')
print(f'Mean Squared Error (MSE): {mse:.10f}')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

linearAgrupado = LinearRegression()

xx_serviciosL = datosModeloAgrupado.drop(['ADR (Native)'], axis=1)
yx_serviciosL = datosModeloAgrupado['ADR (Native)']

X_train_serviciosL, X_test_serviciosL, y_train_serviciosL, y_test_serviciosL = train_test_split(xx_serviciosL, yx_serviciosL, test_size=0.20, random_state=123)

linearAgrupado.fit(X_train_serviciosL, y_train_serviciosL)

y_predL = linearAgrupado.predict(X_test_serviciosL)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_serviciosL, y_predL, alpha=0.7)
plt.plot([min(y_test_serviciosL), max(y_test_serviciosL)], [min(y_test_serviciosL), max(y_test_serviciosL)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

In [ ]:
linearAgrupado.score(X_test_serviciosL, y_test_serviciosL)

y_predL = linearAgrupado.predict(X_test_serviciosL)

mse = mean_squared_error(y_test_serviciosL, y_predL)
mae = mean_absolute_error(y_test_serviciosL, y_predL)
rmse = sqrt(mean_squared_error(y_test_serviciosL, y_predL))
r2 = r2_score(y_test_serviciosL, y_predL)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

In [ ]:
agrupando2 = pd.read_csv('/content/drive/MyDrive/Sergio Naranjo Fernández/DATOS/df_ultimos_12_meses_revenue.csv')
agrupando_alta = agrupando2[(agrupando2['Mes'] >= 6) & (agrupando2['Mes'] <= 8)]
agrupando_alta

agrupando_alta = agrupando_alta.drop(columns=['Precio/Habitacion', 'Number of Reservations', 'Occupancy Rate', 'Revenue (Native)', 'Reservation Days', 'Revenue/Bed', 'Listing Type_2', 'Property Type_2', 'Mes', 'Temporada', 'Reporting Month'])
agrupando_alta.info()

agg_funs = {
    'ADR (Native)': 'mean',
    'Bedrooms' : 'first',
    'Bathrooms' : 'first',
    'Max Guests' : 'first',
    'Extra People Fee (USD)' : 'first',
    'Number of Photos' : 'first',
    'Minimum Stay_x' : 'first',
    'Pets Allowed' : 'first',
    'Pool' : 'first',
    'Lift/Elevator' : 'first',
    'Parking' : 'first',
    'Internet' : 'first',
    'TV' : 'first',
    'Private Pool': 'first',
    'Air' : 'first',
    'dishwasher' : 'first',
    'doorman_entry' : 'first',
    'ENTERTAINMENT_ALL_AGES|Books': 'first',
    'fireplace' : 'first',
    'Bath/Bed' : 'first',
    'Bed/Bath' : 'first',
    'Listing Type' : 'first',
    'Property Type' : 'first'

}

ultimo_final_agg_alta = agrupando_alta.groupby('Property ID').agg(agg_funs).reset_index()
ultimo_final_agg_alta

In [ ]:
ultimo_final_agg_alta_one_hot = pd.get_dummies(ultimo_final_agg_alta, columns=['Listing Type'])

In [ ]:
ultimo_final_agg_alta_one_hot2 = pd.get_dummies(ultimo_final_agg_alta_one_hot, columns=['Property Type'])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

randomAgrupadoAlta = RandomForestRegressor(n_estimators = 100,
            criterion    = 'squared_error',
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,
            random_state = 123)

datosModeloAgrupadoAlta = ultimo_final_agg_alta_one_hot2
datosModeloAgrupadoAlta = datosModeloAgrupadoAlta.drop(columns=['Property ID'])

xx_serviciosRAA = datosModeloAgrupadoAlta.drop(['ADR (Native)'], axis=1)
yx_serviciosRAA = datosModeloAgrupadoAlta['ADR (Native)']


X_train_serviciosRAA, X_test_serviciosRAA, y_train_serviciosRAA, y_test_serviciosRAA = train_test_split(xx_serviciosRAA, yx_serviciosRAA, test_size=0.2, random_state=123)
randomAgrupadoAlta.fit(X_train_serviciosRAA, y_train_serviciosRAA)

y_predRAA = randomAgrupadoAlta.predict(X_test_serviciosRAA)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_serviciosRAA, y_predRAA, alpha=0.7)
plt.plot([min(y_test_serviciosRAA), max(y_test_serviciosRAA)], [min(y_test_serviciosRAA), max(y_test_serviciosRAA)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

In [ ]:
randomAgrupadoAlta.score(X_test_serviciosRAA, y_test_serviciosRAA)

y_predRAA = randomAgrupadoAlta.predict(X_test_serviciosRAA)

mse = mean_squared_error(y_test_serviciosRAA, y_predRAA)
mae = mean_absolute_error(y_test_serviciosRAA, y_predRAA)
rmse = sqrt(mean_squared_error(y_test_serviciosRAA, y_predRAA))
r2 = r2_score(y_test_serviciosRAA, y_predRAA)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

gb_modelAgrupadoAlta = GradientBoostingRegressor(loss='squared_error',
                                     learning_rate=0.1,
                                     n_estimators=100,
                                     subsample=1.0,
                                     criterion='friedman_mse',
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     min_weight_fraction_leaf=0.0,
                                     max_depth=3,
                                     random_state=123)


xx_serviciosGAA = datosModeloAgrupadoAlta.drop(['ADR (Native)'], axis=1)
yx_serviciosGAA = datosModeloAgrupadoAlta['ADR (Native)']

X_train_serviciosGAA, X_test_serviciosGAA, y_train_serviciosGAA, y_test_serviciosGAA = train_test_split(xx_serviciosGAA, yx_serviciosGAA, test_size=0.15, random_state=123)

scaler = StandardScaler()
X_train_servicios_scaledGAA = scaler.fit_transform(X_train_serviciosGAA)
X_test_servicios_scaledGAA = scaler.transform(X_test_serviciosGAA)

gb_modelAgrupadoAlta.fit(X_train_servicios_scaledGAA, y_train_serviciosGAA)

y_predGAA = gb_modelAgrupadoAlta.predict(X_test_servicios_scaledGAA)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_serviciosGAA, y_predGAA, alpha=0.7)
plt.plot([min(y_test_servicios), max(y_test_serviciosGAA)], [min(y_test_serviciosGAA), max(y_test_serviciosGAA)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

mae = mean_absolute_error(y_test_serviciosGAA, y_predGAA)
mse = mean_squared_error(y_test_serviciosGAA, y_predGAA)
rmse = mean_squared_error(y_test_serviciosGAA, y_predGAA, squared=False)  # Aquí calculamos directamente el RMSE usando squared=False
r2 = r2_score(y_test_servicios, y_predGAA)

# Mostrar los resultados
print(f'Mean Absolute Error (MAE): {mae:.10f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.10f}')
print(f'R^2 Score: {r2:.10f}')
print(f'Mean Squared Error (MSE): {mse:.10f}')


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

linearAgrupadoAlta = LinearRegression()
datosModeloAgrupadoAlta = ultimo_final_agg_alta_one_hot2
datosModeloAgrupadoAlta = datosModeloAgrupadoAlta.drop(columns=['Property ID'])

xx_serviciosLAA = datosModeloAgrupadoAlta.drop(['ADR (Native)'], axis=1)
yx_serviciosLAA = datosModeloAgrupadoAlta['ADR (Native)']

X_train_serviciosLAA, X_test_serviciosLAA, y_train_serviciosLAA, y_test_serviciosLAA = train_test_split(xx_serviciosLAA, yx_serviciosLAA, test_size=0.20, random_state=123)

linearAgrupadoAlta.fit(X_train_serviciosLAA, y_train_serviciosLAA)

y_predLAA = linearAgrupadoAlta.predict(X_test_serviciosLAA)

plt.figure(figsize=(10, 6))
plt.scatter(y_test_serviciosLAA, y_predLAA, alpha=0.7)
plt.plot([min(y_test_serviciosLAA), max(y_test_serviciosLAA)], [min(y_test_serviciosLAA), max(y_test_serviciosLAA)], '--r')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Comparación de Predicciones vs Valores Reales')
plt.show()

In [ ]:
linearAgrupadoAlta.score(X_test_serviciosLAA, y_test_serviciosLAA)

y_predLAA = linearAgrupadoAlta.predict(X_test_serviciosLAA)

mse = mean_squared_error(y_test_serviciosLAA, y_predLAA)
mae = mean_absolute_error(y_test_serviciosLAA, y_predLAA)
rmse = sqrt(mean_squared_error(y_test_serviciosLAA, y_predLAA))
r2 = r2_score(y_test_serviciosLAA, y_predLAA)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

In [ ]:
fila = ultimo_final_agg_alta.sample(n=1)
fila

In [ ]:
fila2 = ultimo_final_agg_baja[ultimo_final_agg_baja['Property ID'] == 'ab-910569626739652703']
fila2